# Overview
Initial data-load code and submit code take from this awesome kernel https://www.kaggle.com/kmader/baseline-u-net-model-part-1 



In [ ]:
BATCH_SIZE = 16#32#64#16#64#128#400#512#64#256#128#64#16
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
#from skimage.util.montage import montage2d as montage
import cv2
import random
from datetime import datetime
import json
import gc

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import torch.backends.cudnn
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision.transforms import ToTensor, Normalize, Compose

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from pathlib import Path

from skimage.morphology import label
from skimage.transform import resize

ship_dir = '../input/tgs-salt-identification-challenge/'#'./drive/My Drive/salt/'#'../input/tgs-salt-identification-challenge/'#../input/airbus-ship-detection/'#../input/tgs-salt-identification-challenge/'#
train_image_dir = os.path.join(ship_dir, 'train')
test_image_dir = os.path.join(ship_dir, 'test')
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def multi_rle_encode(img):
    labels = label(img)
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

def rle_decode(mask_rle, shape=(101, 101)):#############################rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction
def upsample(img,img_size):
    return cv2.resize(img.squeeze(), (img_size, img_size), interpolation=cv2.INTER_LINEAR)
#cv2.resize(img, (img_size, img_size))#, mode='constant', preserve_range=True)

def upsamplearray(arr,img_size):
    out=[upsample(x.squeeze(),img_size) for x in arr]
    return np.array(out)    

def downsample(img,img_size):
    #img_size=101
    return cv2.resize(img.squeeze(), (img_size, img_size), interpolation=cv2.INTER_LINEAR)
#resize(img, (img_size, img_size), mode='constant', preserve_range=True)

def downsamplearray(arr):
    out=[downsample(x.squeeze(),101) for x in arr]
    return np.array(out).reshape(-1,101,101,1)

def restore(img):
    #return downsample(img.squeeze()[11:203,11:203])
    img = downsample(img,128)
    return img.squeeze()[13:114,13:114]#cv2.resize(img.squeeze(), (101, 101), interpolation=cv2.INTER_LINEAR)#img.squeeze()[13:114,13:114]#

def flpadding(img,a,b):#13,14
    m,n=img.shape
    output=np.zeros([m+a+b,n+a+b])#([127,127])
    output[a:m+a,a:n+a]=img.squeeze()
    imglr=np.fliplr(img.squeeze())
    output[a:m+a,0:a]=imglr[:,-a:]
    #output[13:114,-13:]=imglr[:,0:13]
    output[a:m+a,-b:]=imglr[:,0:b]
    imgud1=np.flipud(output)
    output[0:a,:]=imgud1[-2*a:-a,:]
    output[-b:,:]=imgud1[b:2*b,:]  
    return output

def imgexpand0(img):
    t_size=128#224
    return upsample(flpadding(img.squeeze(),13,14),t_size).reshape(t_size,t_size,1)
    #return flpadding(img.squeeze(),13,14).reshape(t_size,t_size,1)
    
def imgexarray0(arr):
    out=[imgexpand0(x) for x in arr]
    return np.array(out)

def imgexpand03(img):
    t_size=128#224
    output0=np.zeros([t_size,t_size,3])#([192,192,3])#([127,127])
    output=imgexpand0(img.squeeze())#np.zeros([128,128])#
    for i in range(3):
        output0[:,:,i]=output.squeeze()
    return output0

def imgexarray03(arr):
    out=[imgexpand03(x) for x in arr]
    return np.array(out)

def imgexpand033(img):
    t_size=128#224
    output0=np.zeros([t_size,t_size,3])#([192,192,3])#([127,127])
    output=imgexpand0(img[:,:,0].squeeze())#np.zeros([128,128])#
    for i in range(3):
        output0[:,:,i]=output.squeeze()
    return output0


def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    #all_masks = np.zeros((768, 768), dtype = np.int16)###################################################
    all_masks = np.zeros((101, 101), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)

def mask_overlay(image, mask, color=(0, 1, 0)):
    """
    Helper function to visualize mask on the top of the image
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0
    img[ind] = weighted_sum[ind]    
    return img

def imshow(img, mask, title=None):
    """Imshow for Tensor."""
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    mask = mask.numpy().transpose((1, 2, 0))
    mask = np.clip(mask, 0, 1)
    fig = plt.figure(figsize = (6,6))
    plt.imshow(mask_overlay(img, mask))
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

In [ ]:
print(os.listdir("../input"))

In [ ]:
from tqdm import tqdm_notebook
from keras.preprocessing.image import load_img


train_df11 = pd.read_csv("../input/tgs-salt-identification-challenge/train.csv", index_col="id", usecols=[0])
depths_df11 = pd.read_csv("../input/tgs-salt-identification-challenge/depths.csv", index_col="id")
#train_df
train_df11 = train_df11.join(depths_df11)
test_df11 = depths_df11[~depths_df11.index.isin(train_df11.index)]





df_pred = pd.read_csv('../input/tgs-salt-identification-challenge/train.csv', index_col=[0])
df_pred.fillna('', inplace=True)
df_pred['suspicious'] = False
i=0
for index, row in df_pred.iterrows():
    encoded_mask = row['rle_mask'].split(' ')
    i=i+1
    mask0 = rle_decode(row['rle_mask'])
    #if i==4:
     #   print(np.array(mask0).shape)
     #   plt.imshow(mask0) #
    ##remove small masks
    #s=sum(sum(mask0))
    #if s<30 and s>0:
    #    df_pred.loc[index,'suspicious'] = True
    
    
    #print(np.array(encoded_mask).shape)
    #print(encoded_mask[1])df_pred.loc[index,'suspicious'] = True
    if (len(encoded_mask) > 1 and len(encoded_mask) < 5 and int(encoded_mask[1]) % 101 == 0 and int(encoded_mask[1]) <= 100*101):
        df_pred.loc[index,'suspicious'] = True
        
img_size_ori = 101
img_size_target = 128#127

train_df0 = pd.read_csv("../input/tgs-salt-identification-challenge/train.csv", index_col="id", usecols=[0])
depths_df0 = pd.read_csv("../input/tgs-salt-identification-challenge/depths.csv", index_col="id")
#train_df
train_df0 = train_df0.join(depths_df0)
test_df0 = depths_df0[~depths_df0.index.isin(train_df0.index)]
train_df0["images"] = [np.array(load_img("../input/tgs-salt-identification-challenge/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df0.index)]
#print(train_df.shape) if ~df_pred.loc[idx,'suspicious']
train_df0['suspicious'] = df_pred['suspicious']
train_df0['rle_mask'] = df_pred['rle_mask']
train_df0["masks"] = [np.array(load_img("../input/tgs-salt-identification-challenge/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df0.index)]

train_df0["coverage"] = train_df0.masks.map(np.sum) / pow(img_size_ori, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df0["coverage_class"] = train_df0.coverage.map(cov_to_class)

In [ ]:
train_df1=train_df0.copy()

In [ ]:
train_df0.reset_index(inplace=True)

train_df=train_df0.copy()
valid_df=train_df0.copy()

In [ ]:
train_df.drop(['images','suspicious','coverage','masks'],axis=1,inplace=True)
valid_df.drop(['images','suspicious','coverage','masks'],axis=1,inplace=True)

In [ ]:
grp = list(train_df.groupby('id'))

In [ ]:
zmin=min([list(m['z'].values) for _,m in grp])
zmax=max([list(m['z'].values) for _,m in grp])
zdif=(zmax[0]-zmin[0])
print(zdif)
print(zmin)

## Dataset class for PyThorch dataloader

In [ ]:
def imgexpand03depth(img,depth):
    t_size=128#224
    output0=np.zeros([t_size,t_size,3])#([192,192,3])#([127,127])
    output=imgexpand0(img.squeeze()).squeeze()#np.zeros([128,128])#
    output0[:,:,0]=output
    output0[:,:,1]=output*depth
    output0[:,:,2]=output*depth
    return output0

In [ ]:
def imgexpand03d(img):
    t_size=128#224
    output0=np.zeros([t_size,t_size,3])#([192,192,3])#([127,127])
    output=imgexpand0(img.squeeze()).squeeze()#np.zeros([128,128])#
    output0[:,:,0]=output
    output0[:,:,1]=output
    output0[:,:,2]=output
    return output0

In [ ]:
def add_depth_channels(image_tensor):
    h, w,_ = image_tensor.shape#size()
    for row, const in enumerate(np.linspace(0, 1, h)):
        image_tensor[row, :,1] = const
    image_tensor[:,:,2] = image_tensor[:,:,0] * image_tensor[:,:,1]
    return image_tensor

def C_add_depth_channels(image_tensor,depth):
    h, w,_ = image_tensor.shape#size()
    for row, const in enumerate(np.linspace(0, 1, h)):
        image_tensor[row, :,1] = const
    image_tensor[:,:,2] = image_tensor[:,:,0] * image_tensor[:,:,1]
    image_tensor[:,:,1] = image_tensor[:,:,1] * depth
    return image_tensor

In [ ]:
class SaltDataset(Dataset):
    def __init__(self, in_df, transform=None, mode='train', zdif=908, zmin=51):
        grp = list(in_df.groupby('id'))
        
        self.image_ids =  [_id for _id, _ in grp] 
        self.z = [m['z'].values for _,m in grp]
        self.zmin = zmin
        self.zdif = zdif
        self.image_masks = [m['rle_mask'].values for _,m in grp]
        self.transform = transform
        self.mode = mode
        self.img_transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_ids)
               
    def __getitem__(self, idx):
        img_file_name = self.image_ids[idx]
        if self.mode == 'train':
            rgb_path = os.path.join(train_image_dir, 'images',img_file_name+'.png')
        elif self.mode == 'valid':
            rgb_path = os.path.join(train_image_dir, 'images',img_file_name+'.png')
        else:
            if img_file_name[-4:]=='.png':
                rgb_path = os.path.join(test_image_dir, 'images',img_file_name)
            else:
                rgb_path = os.path.join(test_image_dir, 'images',img_file_name+'.png')
        img = imread(rgb_path)
        mask = masks_as_image(self.image_masks[idx])  
        depth = self.z[idx]
        depth = math.ceil((depth-self.zmin)/self.zdif*101.)/101.
        if depth==0:
            depth=1./101.
        #print(depth)
        if self.transform is not None:
            img, mask = self.transform(img, mask)
            
        img = imgexpand03d(img[:,:,0])
        img = add_depth_channels(img)#
        size0=192
        img = cv2.resize(img[:, :, :], (size0, size0), interpolation=cv2.INTER_LINEAR)
        
        mask =imgexpand0(mask)#imgexpand256(mask)###################################################################
        mask = cv2.resize(mask, (size0, size0), interpolation=cv2.INTER_LINEAR)    
        
        if mask is not None:           
            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=2)            
        if img is not None:           
            if img.ndim == 2:
                img = np.expand_dims(img, axis=2)                   
                
        if self.mode == 'train':
            #return self.to_float_tensor(img), self.to_float_tensor(mask)
            #eturn img, mask
            #print(mask.shape)
            return self.img_transform(img), torch.from_numpy(np.moveaxis(mask, -1, 0)).float()
        else:
            return self.img_transform(img), str(img_file_name)

## Check some image from validation dataset

In [ ]:
#Create dataset
#dataset_valid = ShipDataset(train_df)
dataset_valid = SaltDataset(valid_df)

#dataset_valid_sus = SaltDataset(valid_df_sus)
#Create dataset
#dataset_valid = ShipDataset(valid_df)

#To show image with its mask
imshow(*dataset_valid[0])
print(*dataset_valid[0])

# Augment Data

We should create custom classes for sumultaneous transformation images and  masks

In [ ]:
"""
    Implementation from  https://github.com/ternaus/robot-surgery-segmentation
"""

def clip(img, dtype, maxval):
    return np.clip(img, 0, maxval).astype(dtype)

class DualCompose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, x, mask=None):
        for t in self.transforms:
            x, mask = t(x, mask)
            if mask is not None:           
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
        return x, mask
    
class OneOf:
    def __init__(self, transforms, prob=0.5):
        self.transforms = transforms
        self.prob = prob

    def __call__(self, x, mask=None):
        if random.random() < self.prob:
            t = random.choice(self.transforms)
            t.prob = 1.
            x, mask = t(x, mask)
            #print(x.shape)
            #print(mask.shape)
            if mask is not None:           
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
        return x, mask

class OneOrOther:
    def __init__(self, first, second, prob=0.5):
        self.first = first
        first.prob = 1.
        self.second = second
        second.prob = 1.
        self.prob = prob

    def __call__(self, x, mask=None):
        if random.random() < self.prob:
            x, mask = self.first(x, mask)
        else:
            x, mask = self.second(x, mask)
        if mask is not None:           
            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=2)
        return x, mask


class ImageOnly:
    def __init__(self, trans):
        self.trans = trans

    def __call__(self, x, mask=None):
        return self.trans(x), mask


class VerticalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            img = cv2.flip(img, 0)
            if mask is not None:
                mask = cv2.flip(mask, 0)
        return img, mask


class HorizontalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            img = cv2.flip(img, 1)
            if mask is not None:
                mask = cv2.flip(mask, 1)
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
        return img, mask


class RandomFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            d = random.randint(-1, 1)
            img = cv2.flip(img, d)
            if mask is not None:
                mask = cv2.flip(mask, d)
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
        return img, mask


class Transpose:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            img = img.transpose(1, 0, 2)
            if mask is not None:
                mask = mask.transpose(1, 0, 2)
        return img, mask


class RandomRotate90:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            factor = random.randint(0, 4)
            img = np.rot90(img, factor)
            if mask is not None:
                mask = np.rot90(mask, factor)
        return img.copy(), mask.copy()


class Rotate:
    def __init__(self, limit=90, prob=0.5):
        self.prob = prob
        self.limit = limit

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            angle = random.uniform(-self.limit, self.limit)

            height, width = img.shape[0:2]
            mat = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1.0)
            img = cv2.warpAffine(img, mat, (height, width),
                                 flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_REFLECT_101)
            if mask is not None:
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
                mask = cv2.warpAffine(mask, mat, (height, width),
                                      flags=cv2.INTER_LINEAR,
                                      borderMode=cv2.BORDER_REFLECT_101)

        return img, mask


class RandomCrop:
    def __init__(self, size):
        self.h = size[0]
        self.w = size[1]

    def __call__(self, img, mask=None):
        height, width, _ = img.shape

        h_start = np.random.randint(0, height - self.h)
        w_start = np.random.randint(0, width - self.w)

        img = img[h_start: h_start + self.h, w_start: w_start + self.w,:]

        assert img.shape[0] == self.h
        assert img.shape[1] == self.w

        if mask is not None:
            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=2)
            mask = mask[h_start: h_start + self.h, w_start: w_start + self.w,:]

        return img, mask


class Shift:
    def __init__(self, limit=4, prob=.5):
        self.limit = limit
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            limit = self.limit
            dx = round(random.uniform(-limit, limit))
            dy = round(random.uniform(-limit, limit))

            height, width, channel = img.shape
            y1 = limit + 1 + dy
            y2 = y1 + height
            x1 = limit + 1 + dx
            x2 = x1 + width

            img1 = cv2.copyMakeBorder(img, limit + 1, limit + 1, limit + 1, limit + 1,
                                      borderType=cv2.BORDER_REFLECT_101)
            img = img1[y1:y2, x1:x2, :]
            if mask is not None:
                msk1 = cv2.copyMakeBorder(mask, limit + 1, limit + 1, limit + 1, limit + 1,
                                          borderType=cv2.BORDER_REFLECT_101)
                mask = msk1[y1:y2, x1:x2, :]

        return img, mask


class ShiftScale:
    def __init__(self, limit=4, prob=.25):
        self.limit = limit
        self.prob = prob

    def __call__(self, img, mask=None):
        limit = self.limit
        if random.random() < self.prob:
            height, width, channel = img.shape
            assert (width == height)
            size0 = width
            size1 = width + 2 * limit
            size = round(random.uniform(size0, size1))

            dx = round(random.uniform(0, size1 - size))
            dy = round(random.uniform(0, size1 - size))

            y1 = dy
            y2 = y1 + size
            x1 = dx
            x2 = x1 + size

            img1 = cv2.copyMakeBorder(img, limit, limit, limit, limit, borderType=cv2.BORDER_REFLECT_101)
            img = (img1[y1:y2, x1:x2, :] if size == size0
            else cv2.resize(img1[y1:y2, x1:x2, :], (size0, size0), interpolation=cv2.INTER_LINEAR))

            if mask is not None:
                msk1 = cv2.copyMakeBorder(mask, limit, limit, limit, limit, borderType=cv2.BORDER_REFLECT_101)
                mask = (msk1[y1:y2, x1:x2, :] if size == size0
                else cv2.resize(msk1[y1:y2, x1:x2, :], (size0, size0), interpolation=cv2.INTER_LINEAR))

        return img, mask


class ShiftScaleRotate:
    def __init__(self, shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, prob=0.5):
        self.shift_limit = shift_limit
        self.scale_limit = scale_limit
        self.rotate_limit = rotate_limit
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            height, width, channel = img.shape

            angle = random.uniform(-self.rotate_limit, self.rotate_limit)
            scale = random.uniform(1 - self.scale_limit, 1 + self.scale_limit)
            dx = round(random.uniform(-self.shift_limit, self.shift_limit)) * width
            dy = round(random.uniform(-self.shift_limit, self.shift_limit)) * height

            cc = math.cos(angle / 180 * math.pi) * scale
            ss = math.sin(angle / 180 * math.pi) * scale
            rotate_matrix = np.array([[cc, -ss], [ss, cc]])

            box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
            box1 = box0 - np.array([width / 2, height / 2])
            box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

            box0 = box0.astype(np.float32)
            box1 = box1.astype(np.float32)
            mat = cv2.getPerspectiveTransform(box0, box1)
            img = cv2.warpPerspective(img, mat, (width, height),
                                      flags=cv2.INTER_LINEAR,
                                      borderMode=cv2.BORDER_REFLECT_101)
            if mask is not None:
                mask = cv2.warpPerspective(mask, mat, (width, height),
                                           flags=cv2.INTER_NEAREST,
                                           borderMode=cv2.BORDER_REFLECT_101)

        return img, mask


class CenterCrop:
    def __init__(self, size):
        self.height = size[0]
        self.width = size[1]

    def __call__(self, img, mask=None):
        h, w, c = img.shape
        dy = (h - self.height) // 2
        dx = (w - self.width) // 2
        y1 = dy
        y2 = y1 + self.height
        x1 = dx
        x2 = x1 + self.width
        img = img[y1:y2, x1:x2,:]
        if mask is not None:
            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=2)
            mask = mask[y1:y2, x1:x2,:]

        return img, mask
    
class RandomBrightness:
    def __init__(self, limit=0.1, prob=0.5):
        self.limit = limit
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            alpha = 1.0 + self.limit * random.uniform(-1, 1)

            maxval = np.max(img[..., :3])
            dtype = img.dtype
            img[..., :3] = clip(alpha * img[..., :3], dtype, maxval)
            if mask is not None:           
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
        return img,mask


class RandomContrast:
    def __init__(self, limit=.1, prob=.5):
        self.limit = limit
        self.prob = prob

    def __call__(self, img):
        if random.random() < self.prob:
            alpha = 1.0 + self.limit * random.uniform(-1, 1)

            gray = cv2.cvtColor(img[:, :, :3], cv2.COLOR_BGR2GRAY)
            gray = (3.0 * (1.0 - alpha) / gray.size) * np.sum(gray)
            maxval = np.max(img[..., :3])
            dtype = img.dtype
            img[:, :, :3] = clip(alpha * img[:, :, :3] + gray, dtype, maxval)
        return img

class Imgexpand:
    def __init__(self, s=101,e=128):
        self.s = s
        self.e = e

    def __call__(self, img, mask):
        output0=np.zeros([128,128,3])#([127,127])
        mask0=np.zeros([128,128,1])#([127,127])
        output=np.zeros([128,128])#
        #for i in range(3)
        output[13:114,13:114]=img[:,:,0]
        imglr=np.fliplr(img[:,:,0])
        output[13:114,0:13]=imglr[:,-13:]
        #output[13:114,-13:]=imglr[:,0:13]
        output[13:114,-14:]=imglr[:,0:14]
        imgud1=np.flipud(output)
        output[0:13,:]=imgud1[-26:-13,:]
        output[-14:,:]=imgud1[14:28,:] 
        #output[-13:,:]=imgud1[13:26,:]  
        #img=output
        for i in range(3):
            output0[:,:,i]=output
            
        output[13:114,13:114]=mask.squeeze()
        imglr=np.fliplr(img[:,:,0])
        output[13:114,0:13]=imglr[:,-13:]
        #output[13:114,-13:]=imglr[:,0:13]
        output[13:114,-14:]=imglr[:,0:14]
        imgud1=np.flipud(output)
        output[0:13,:]=imgud1[-26:-13,:]
        output[-14:,:]=imgud1[14:28,:]
        mask0[:,:,-1]=output
        return output0,mask0
    
class ZoominRandomCrop:
    def __init__(self, size, scale, prob=0.5):
        self.h = size[0]#101
        self.w = size[1]#101
        self.s = scale
        self.prob = prob
        #self.ori=ori
    def __call__(self, img, mask=None):
        #img = img[13: 114, 13: 114,:]
        if random.random() < self.prob: 
            #y1=13
            #y2=114
            #x1=13
            #x2=114
            hw=int(self.h*self.s)
            img1=cv2.resize(img, (hw, hw), interpolation=cv2.INTER_LINEAR)
            #img1=cv2.resize(img[y1:y2, x1:x2, :], (hw, hw), interpolation=cv2.INTER_LINEAR)
            #height, width, _ = 120#img.shape
            height =hw
            width  =hw

            h_start = np.random.randint(0, height - self.h)
            w_start = np.random.randint(0, width - self.w)

            img = img1[h_start: h_start + self.h, w_start: w_start + self.w,:]
            #img = imgexpand033(img1)

            #assert img.shape[0] == self.h
            #assert img.shape[1] == self.w

            if mask is not None:
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
                #print(mask.shape)
                mask1=cv2.resize(mask, (hw, hw), interpolation=cv2.INTER_LINEAR)
                if mask1.ndim == 2:
                    mask1 = np.expand_dims(mask1, axis=2)                
                #print(mask1.shape)
                mask = mask1[h_start: h_start + self.h, w_start: w_start + self.w,:]

        return img, mask
    
class ZoominRandomCropNew:
    def __init__(self, size, scale, prob=0.5):
        self.h = size[0]#101
        self.w = size[1]#101
        self.s = scale
        self.prob = prob
        #self.ori=ori
    def __call__(self, img, mask=None):
        #img = img[13: 114, 13: 114,:]
        if random.random() < self.prob: 
            #y1=13
            #y2=114
            #x1=13
            #x2=114
            #hw=int(self.h*self.s)
            sc=random.random()*(self.s-1)+1
            hw=int(self.h*sc)+1
            img1=cv2.resize(img, (hw, hw), interpolation=cv2.INTER_LINEAR)
            #img1=cv2.resize(img[y1:y2, x1:x2, :], (hw, hw), interpolation=cv2.INTER_LINEAR)
            #height, width, _ = 120#img.shape
            height =hw
            width  =hw
            #print(hw)
            h_start = np.random.randint(0, height - self.h)
            w_start = np.random.randint(0, width - self.w)

            img = img1[h_start: h_start + self.h, w_start: w_start + self.w,:]
            #img = imgexpand033(img1)

            #assert img.shape[0] == self.h
            #assert img.shape[1] == self.w

            if mask is not None:
                if mask.ndim == 2:
                    mask = np.expand_dims(mask, axis=2)
                #print(mask.shape)
                mask1=cv2.resize(mask, (hw, hw), interpolation=cv2.INTER_LINEAR)
                if mask1.ndim == 2:
                    mask1 = np.expand_dims(mask1, axis=2)                
                #print(mask1.shape)
                mask = mask1[h_start: h_start + self.h, w_start: w_start + self.w,:]

        return img, mask

In [ ]:
train_transform = DualCompose([
        HorizontalFlip(),
        ZoominRandomCropNew([101,101],1.10),
        Rotate(limit=5),
        RandomBrightness(limit=0.08),
])

val_transform = DualCompose([
        #Imgexpand(),##############################################CenterCrop((512,512,3)),
      ])

# UNET model

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models
import torch.nn.functional as F
import math
import torch.utils.model_zoo as model_zoo

nonlinearity = nn.ReLU
class EncoderBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride):
        super().__init__()
        
        self.c1=nn.Conv2d(inchannel, outchannel, 3, stride, 1, bias=False)
        self.bn1=nn.BatchNorm2d(outchannel)
        self.re1=nn.ReLU(inplace=True)
        self.c2=nn.Conv2d(outchannel, outchannel, 3, 1, 1, bias=False)
        self.bn2=nn.BatchNorm2d(outchannel)
        self.re2=nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.c1(x)
        x = self.bn1(x)
        x = self.re1(x)
        x = self.c2(x)
        x = self.bn2(x)
        x = self.re2(x)
        return x
class EncoderBlock0(nn.Module):
    def __init__(self, in_channels, n_filters):
        super().__init__()

        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C/4, H, W
        self.pool = nn.MaxPool2d(2, 2)#
        self.norm2 = nn.BatchNorm2d(in_channels // 4)
        self.relu2 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)
        self.relu3 = nonlinearity(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.pool(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        return x
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, n_filters):
        super().__init__()

        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C/4, H, W
        self.deconv2 = nn.ConvTranspose2d(in_channels // 4, in_channels // 4, 3,
                                          stride=2, padding=1, output_padding=1)
        self.norm2 = nn.BatchNorm2d(in_channels // 4)
        self.relu2 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)
        self.relu3 = nonlinearity(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.deconv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        return x

class ChannelSE(nn.Module):
    def __init__(self,inchannel):
        super().__init__()
        self.lin1=torch.nn.Linear(inchannel, inchannel//2)
        self.lin2=torch.nn.Linear(inchannel//2, inchannel)
        self.c=inchannel
    def forward(self,x):
        #_,c,h,w=x.size
        #print(c)
        #print(h)
        #print(w)
        m=torch.mean(torch.mean(x,dim=2,keepdim=True),dim=3,keepdim=True)
        m = m.view(m.size(0), -1)
        m=self.lin1(m)
        m=nn.ReLU()(m)
        m=self.lin2(m)

        m=nn.Sigmoid()(m)
        m = m.view(m.size(0), self.c,1,1)
        x=m*x#torch.matmul(m,x)
        return x

class SpatialSE(nn.Module):
    def __init__(self,inchannel):
        super().__init__()
        self.conv=torch.nn.Conv2d(inchannel,1,kernel_size=1,stride=1)
    def forward(self,x):
        #_,c,h,w=x.size
        #print(c)
        #print(h)
        #print(w)
        m = self.conv(x)
        m=nn.Sigmoid()(m)
        x=m*x#torch.matmul(m,x)
        return x

class DecoderBlockv(nn.Module):
    def __init__(self, in_channels, n_filters):
        super().__init__()

        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C/4, H, W
        self.deconv2 = nn.ConvTranspose2d(in_channels // 4, in_channels // 4, 3,
                                          stride=2, padding=1, output_padding=1)
        self.norm2 = nn.BatchNorm2d(in_channels // 4)
        self.relu2 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)
        self.relu3 = nonlinearity(inplace=True)
        
        self.cSE = ChannelSE(n_filters)
        self.sSE = SpatialSE(n_filters)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.deconv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        x = self.cSE(x) + self.sSE(x)
        return x
class ConvUp(nn.Module):
    def __init__(self, in_channels, n_filters):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2,mode='bilinear')
        self.conv1 = nn.Conv2d(in_channels, n_filters, 3, padding = 1)
        self.norm1 = nn.BatchNorm2d(n_filters)
        self.relu1 = nonlinearity(inplace=True)

    def forward(self, x):
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        return x   
class ConscSE(nn.Module):
    def __init__(self, n_filters):
        super().__init__()

        
        self.cSE = ChannelSE(n_filters)
        self.sSE = SpatialSE(n_filters)
        
    def forward(self, x):
        x = self.cSE(x) + self.sSE(x)
        return x
    
class DecoderBlockup(nn.Module):
    def __init__(self, in_channels, n_filters):
        super().__init__()

        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C/4, H, W
        self.deconv2 = ConvUp(in_channels // 4, in_channels // 4)
        self.norm2 = nn.BatchNorm2d(in_channels // 4)
        self.relu2 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)
        self.relu3 = nonlinearity(inplace=True)
        
        self.cSE = ChannelSE(n_filters)
        self.sSE = SpatialSE(n_filters)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.deconv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        x = self.cSE(x) + self.sSE(x)
        return x
    
class DecoderBlock23(nn.Module):
    def __init__(self, in_channels, n_filters, scal=4):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2,mode='bilinear')
        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, in_channels // scal, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // scal)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv2 = nn.Conv2d(in_channels // scal, n_filters, 1)
        self.norm2 = nn.BatchNorm2d(n_filters)
        self.relu2 = nonlinearity(inplace=True)
        
        self.cSE = ChannelSE(n_filters)
        self.sSE = SpatialSE(n_filters)
        
    def forward(self, x):
        x = self.up(x)
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        #x = self.cSE(x) + self.sSE(x)
        return x
    
class Upscale:
    transposed_conv = 0
    upsample_bilinear = 1
    pixel_shuffle = 2  
class BasicDecoderBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels, conv_size=3, upscale=Upscale.transposed_conv):
        super().__init__()
        padding = 0
        if conv_size == 3:
            padding = 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, middle_channels, conv_size, padding=padding),
            nn.BatchNorm2d(middle_channels),
            nn.ReLU(inplace=True)
        )

        last_conv_channels = middle_channels
        if upscale == Upscale.transposed_conv:
            self.layer2 = nn.Sequential(
                nn.ConvTranspose2d(middle_channels, middle_channels, 3, stride=2, padding=1, output_padding=1),
                nn.BatchNorm2d(middle_channels),
                nn.ReLU(inplace=True)
            )
        elif upscale == Upscale.upsample_bilinear:
            self.layer2 = nn.Upsample(scale_factor=2)
        else:
            self.layer2 = nn.PixelShuffle(upscale_factor=2)
            last_conv_channels = middle_channels // 4

        self.layer3 = nn.Sequential(
            nn.Conv2d(last_conv_channels, out_channels, conv_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

class UnetBNDecoderBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels, upscale=Upscale.upsample_bilinear):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.layer(x)
    
class LinkNet34a(nn.Module):
    def __init__(self, num_classes, num_channels=3):
        super().__init__()
        assert num_channels == 3, "num channels not used now. to use changle first conv layer to support num channels other then 3"
        filters = [64, 128, 256, 512]
        resnet = models.resnet34(pretrained=True)

        self.firstconv = resnet.conv1
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3
        self.encoder4 = resnet.layer4
        # Center
        self.center = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.Conv2d(filters[3], filters[1], 3, padding=1),
            nn.BatchNorm2d(filters[1]),
            nn.ReLU(inplace=True)            
        
        )
        # Decoder
        self.decoder5 = UnetBNDecoderBlock(filters[1],filters[2]//4, filters[2])#
        self.conv5=nn.Conv2d(256+512,256,1) 
        self.decoder4 = UnetBNDecoderBlock(filters[2],filters[2]//4, filters[1])#DecoderBlock(filters[3], filters[2])
        self.conv4=nn.Conv2d(128+256,256,1)        
        self.decoder3 = UnetBNDecoderBlock(filters[2],filters[2]//4, filters[0])#DecoderBlock(filters[2], filters[1])
        self.conv3=nn.Conv2d(64+128,128,1)        
        self.decoder2 = UnetBNDecoderBlock(filters[1],filters[1]//4, filters[0])#DecoderBlock(filters[1], filters[0])
        self.conv2=nn.Conv2d(128,64,1)        
        #self.decoder1 = UnetBNDecoderBlock(filters[0],filters[0]//4, filters[0])#DecoderBlock(filters[0], filters[0])

        # Final Classifier
        self.finaldeconv1 = UnetBNDecoderBlock(filters[0],filters[0]//4, filters[0])#ConvUp(filters[0], filters[0])
        
        # Final Classifier
        self.logit = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nonlinearity(inplace=True),
            nn.Conv2d(64, 1, 1),
        )        


    # noinspection PyCallingNonCallable
    def forward(self, x):
        # Encoder
        x = x.float()
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        #x = self.firstmaxpool(x)
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        ############################
        e5 = self.center(e4)
        d5 = torch.cat([self.decoder5(e5) , e4], 1)#concat([self.decoder5(e5) , e4])
        d5 = self.conv5(d5)     
        #########################
        d4 = torch.cat([self.decoder4(d5) , e3], 1)#concat([self.decoder5(e5) , e4])
        d4 = self.conv4(d4)
        # d4 = e3
        #d3 = self.decoder3(d4) + e2
        #print(e2.shape)
        d3 = torch.cat([self.decoder3(d4) , e2], 1)#concat([self.decoder5(e5) , e4])
        #print(d3.shape)
        d3 = self.conv3(d3)
        
        #d2 = self.decoder2(d3) + e1
        d2 = torch.cat([self.decoder2(d3) , e1], 1)#concat([self.decoder5(e5) , e4])
        d2 = self.conv2(d2)
        
        #d1 = self.decoder1(d2)

        # Final Classification
        f = self.finaldeconv1(d2)
        #f = self.finalrelu1(f)
        f = self.logit(f)
        return f

In [ ]:
class DecoderBlockH(nn.Module):
    def __init__(self, in_channels,channels, n_filters):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2,mode='bilinear')
        
        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, channels, 3, padding=1)
        self.norm1 = nn.BatchNorm2d(channels)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv2 = nn.Conv2d(channels, n_filters, 3, padding=1)
        self.norm2 = nn.BatchNorm2d(n_filters)
        self.relu2 = nonlinearity(inplace=True)
        
        #self.cSE = ChannelSE(n_filters)
        #self.sSE = SpatialSE(n_filters)
        
    def forward(self, x, e=None):
        x = self.up(x)
        if e is not None:
            x = torch.cat([x, e], 1)
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        #x = self.cSE(x) + self.sSE(x)
        return x

In [ ]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        """Global average pooling over the input's spatial dimensions"""
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, inputs):
        in_size = inputs.size()
        inputs = inputs.view((in_size[0], in_size[1], -1)).mean(dim=2)
        return inputs.view((in_size[0], in_size[1], 1, 1))







class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.fcs = nn.Sequential(nn.Linear(channel, int(channel/reduction)),
                                 nn.LeakyReLU(negative_slope=0.1, inplace=True),
                                 nn.Linear(int(channel/reduction), channel),
                                 nn.Sigmoid())

    def forward(self, x):
        bahs, chs, _, _ = x.size()

        # Returns a new tensor with the same data as the self tensor but of a different size.
        y = self.avg_pool(x).view(bahs, chs)
        y = self.fcs(y).view(bahs, chs, 1, 1)
        return torch.mul(x, y)


class SCSEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SCSEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.channel_excitation = nn.Sequential(nn.Linear(channel, int(channel//reduction)),
                                                nn.ReLU(inplace=True),
                                                nn.Linear(int(channel//reduction), channel),
                                                nn.Sigmoid())

        self.spatial_se = nn.Sequential(nn.Conv2d(channel, 1, kernel_size=1,
                                                  stride=1, padding=0, bias=False),
                                        nn.Sigmoid())

    def forward(self, x):
        bahs, chs, _, _ = x.size()

        # Returns a new tensor with the same data as the self tensor but of a different size.
        chn_se = self.avg_pool(x).view(bahs, chs)
        chn_se = self.channel_excitation(chn_se).view(bahs, chs, 1, 1)
        chn_se = torch.mul(x, chn_se)

        spa_se = self.spatial_se(x)
        spa_se = torch.mul(x, spa_se)
        return torch.add(chn_se, 1, spa_se)


class ModifiedSCSEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(ModifiedSCSEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.channel_excitation = nn.Sequential(nn.Linear(channel, int(channel//reduction)),
                                                nn.ReLU(inplace=True),
                                                nn.Linear(int(channel//reduction), channel),
                                                nn.Sigmoid())

        self.spatial_se = nn.Sequential(nn.Conv2d(channel, 1, kernel_size=1,
                                                  stride=1, padding=0, bias=False),
                                        nn.Sigmoid())

    def forward(self, x):
        bahs, chs, _, _ = x.size()

        # Returns a new tensor with the same data as the self tensor but of a different size.
        chn_se = self.avg_pool(x).view(bahs, chs)
        chn_se = self.channel_excitation(chn_se).view(bahs, chs, 1, 1)

        spa_se = self.spatial_se(x)
        return torch.mul(torch.mul(x, chn_se), spa_se) 
    

In [ ]:
class ConvBn2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.layer = nn.Sequential(
            #nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels, out_channels, kernel_size, padding),
            nn.BatchNorm2d(out_channels),
            #nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.layer(x)
    
class Decoder3(nn.Module):
    def __init__(self, in_channels,res_channels, channels, n_filters):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear')
        
        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels+res_channels, channels, 3, padding=1)
        self.norm1 = nn.BatchNorm2d(channels)
        self.relu1 = nonlinearity(inplace=True)

        # B, C/4, H, W -> B, C, H, W
        self.conv2 = nn.Conv2d(channels, n_filters, 3, padding=1)
        self.norm2 = nn.BatchNorm2d(n_filters)
        self.relu2 = nonlinearity(inplace=True)
        
        self.SCSE = SCSEBlock(n_filters)#ChannelSE(n_filters)
        #self.sSE = SpatialSE(n_filters)
        
    def forward(self, x, e=None):
        x = self.up(x)
        if e is not None:
            x = torch.cat([x, e], 1)
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.SCSE(x)# + self.sSE(x)
        return x
    
class DenseNet34(nn.Module):
    def __init__(self ):
        super().__init__()
        #super(Net,self).__init__()
        filters = [64, 128, 256, 512]
        self.resnet =  models.resnet34(pretrained=True)#ResNet(BasicBlock, [3, 4, 6, 3], num_classes=1 )

        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            self.resnet.layer1,
        )
        self.encoder3 = self.resnet.layer2
        self.encoder4 = self.resnet.layer3
        self.encoder5 = self.resnet.layer4

        self.center = nn.Sequential(
            ConvBn2d( 512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ConvBn2d( 256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
######################################################################
        #self.decoder5 = Decoder3(256, 512, 512, 64)
        #self.decoder4 = Decoder3( 64, 256, 256, 64)
        #self.decoder3 = Decoder3( 64, 128, 128, 64)
        #self.decoder2 = Decoder3( 64,  64,  64, 64)
        #self.decoder1 = Decoder3( 64,  64,  32, 64)
        
        self.decoder5 = DecoderBlockH(filters[3]+filters[2],filters[2], 64)
        #self.conv5=nn.Conv2d(64+512,64,1)#before or after SE?
        self.se5=SCSEBlock(64)
        self.decoder4 = DecoderBlockH(filters[2]+64, filters[1], 64)
        #self.conv4=nn.Conv2d(64+256,64,1)        
        self.se4=SCSEBlock(64)
        self.decoder3 = DecoderBlockH(filters[1]+64, filters[1], 64)
        #self.conv3=nn.Conv2d(64+128,64,1)    
        self.se3=SCSEBlock(64)
        self.decoder2 = DecoderBlockH(filters[0]+64, filters[0], 64)
        #self.conv2=nn.Conv2d(64+64,64,1)   
        self.se2=SCSEBlock(64)
        self.decoder1 = DecoderBlockH(filters[0], filters[0]//2, 64)
        self.se1=SCSEBlock(64)        
        
##############################################################################        
        self.fuse_pixel  = nn.Sequential(        
            nn.Conv2d(320, 64, kernel_size=3, padding=1),
        )
        self.logit_pixel  = nn.Sequential(
            #nn.Conv2d(320, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d( 64,  1, kernel_size=1, padding=0),
        )

        self.logit_image = nn.Sequential(
            #nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            #nn.Linear(128, 1),
            nn.Linear(64, 1),
        )
        self.fuse_image = nn.Sequential(
            nn.Linear(512, 64),
            #nn.ReLU(inplace=True),
            #nn.Linear(128, 1),
        )
        self.fuse = nn.Sequential(
            #nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            #nn.Conv2d(128, 64, kernel_size=1, padding=0),
            #nn.BatchNorm2d(64),
            #nn.ReLU(inplace=True),
        )
        self.logit = nn.Sequential(
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
            #nn.ReLU(inplace=True),
            #nn.Conv2d( 64,  1, kernel_size=1, padding=0),
        )

    def forward(self, x):
        batch_size,C,H,W = x.shape
        """
        mean=[0.485, 0.456, 0.406]
        std =[0.229, 0.224, 0.225]
        x = torch.cat([
            (x-mean[2])/std[2],
            (x-mean[1])/std[1],
            (x-mean[0])/std[0],
        ],1)
        """
        x = x.float()

        e1 = self.encoder1(x )  #; print('e1',e1.size())
        e2 = self.encoder2(e1)  #; print('e2',e2.size())
        e3 = self.encoder3(e2)  #; print('e3',e3.size())
        e4 = self.encoder4(e3)  #; print('e4',e4.size())
        e5 = self.encoder5(e4)  #; print('e5',e5.size())

        f = self.center(e5)                #; print('f',f.size())
        #print(f.shape)
        #print(e5.shape)
        #e1 = self.encoder1(x)#
        #e2 = self.encoder2(e1)#
        #e3 = self.encoder3(e2)#
        #e4 = self.encoder4(e3)#

        #e5 = self.center(e4)#512
        ####################################################################################
        #d5 = self.decoder5( f,e5)          #; print('d5',f.size())
        #d4 = self.decoder4(d5,e4)          #; print('d4',f.size())
        #d3 = self.decoder3(d4,e3)          #; print('d3',f.size())
        #d2 = self.decoder2(d3,e2)          #; print('d2',f.size())
        #d1 = self.decoder1(d2,e1)          #; print('d1',f.size())
        
        d5 = self.decoder5(f,e5)
        d5 = self.se5(d5)
        # Decoder with Skip Connections
        #d4 = self.decoder4(d5) + e3
        #d4 = torch.cat([self.decoder4(d5) , e3], 1)#concat([self.decoder5(e5) , e4])
        #print(d5.shape)
        #print(e3.shape)
        d4 = self.decoder4(d5,e4)
        d4 = self.se4(d4)
        # d4 = e3
        #d3 = self.decoder3(d4) + e2
        #print(e2.shape)
        #d3 = torch.cat([self.decoder3(d4) , e2], 1)#concat([self.decoder5(e5) , e4])
        #print(d3.shape)
        d3 = self.decoder3(d4,e3)
        d3 = self.se3(d3)
        
        #d2 = self.decoder2(d3) + e1
        #d2 = torch.cat([self.decoder2(d3) , e1], 1)#concat([self.decoder5(e5) , e4])
        d2 = self.decoder2(d3,e2)
        d2 = self.se2(d2)
        d1 = self.decoder1(d2)
        d1 = self.se1(d1)
        ########################################################################################
        d = torch.cat((
            d1,
            F.upsample(d2,scale_factor= 2, mode='bilinear',align_corners=False),
            F.upsample(d3,scale_factor= 4, mode='bilinear',align_corners=False),
            F.upsample(d4,scale_factor= 8, mode='bilinear',align_corners=False),
            F.upsample(d5,scale_factor=16, mode='bilinear',align_corners=False),
        ),1)
        
        #######################################################################
        """
        d = F.dropout(d, p=0.50, training=self.training)
        logit_pixel = self.logit_pixel(d)

        f = F.adaptive_avg_pool2d(e5, output_size=1).view(batch_size,-1)
        f = F.dropout(f, p=0.50, training=self.training)
        logit_image = self.logit_image(f).view(-1)
        """
        ###########################################################################
        #d = torch.cat([d1,d2,d3,d4,d5],1) #hyper-columns
        d = F.dropout(d, p=0.50, training=self.training)
        fuse_pixel  = self.fuse_pixel(d)#64-128-128
        logit_pixel = self.logit_pixel(fuse_pixel)#1-128-128

        e = F.adaptive_avg_pool2d(e5, output_size=1).view(batch_size,-1) #image pool#-512-1-1
        e = F.dropout(e, p=0.50, training=self.training)#
        fuse_image  = self.fuse_image(e)#-64-1-1
        logit_image = self.logit_image(fuse_image).view(-1)#-1-1-1

        #fuse = self.fuse(torch.mul(fuse_pixel, F.upsample(fuse_image.view(batch_size,-1,1,1,),scale_factor=128, mode='nearest')))
        #fuse = self.fuse(fuse_pixel+ F.upsample(fuse_image.view(batch_size,-1,1,1,),scale_factor=128, mode='nearest'))
        fuse = self.fuse(torch.cat([ #fuse
            fuse_pixel,
            F.upsample(fuse_image.view(batch_size,-1,1,1,),scale_factor=128, mode='nearest')
        ],1))
        logit = self.logit(fuse)#1-128-128

        return logit, logit_pixel, logit_image        
        
        
        #return logit_pixel, logit_image


    ##-----------------------------------------------------------------


    #def criterion(self, logit_pixel, logit_image, truth_pixel, truth_image, is_average=True):

    
    
    
"""
d3 = F.upsample(d3,scale_factor= 4, mode='bilinear',align_corners=False)
    d4 = F.upsample(d4,scale_factor= 8, mode='bilinear',align_corners=False)
    d5 = F.upsample(d5,scale_factor=16, mode='bilinear',align_corners=False)

    d = torch.cat([d1,d2,d3,d4,d5],1) #hyper-columns
    d = F.dropout(d, p=0.50, training=self.training)
    fuse_pixel  = self.fuse_pixel(d)
    logit_pixel = self.logit_pixel(fuse_pixel)

    e = F.adaptive_avg_pool2d(e5, output_size=1).view(batch_size,-1) #image pool
    e = F.dropout(e, p=0.50, training=self.training)
    fuse_image  = self.fuse_image(e)
    logit_image = self.logit_image(fuse_image).view(-1)

    fuse = self.fuse(torch.cat([ #fuse
        fuse_pixel,
        F.upsample(fuse_image.view(batch_size,-1,1,1,),scale_factor=128, mode='nearest')
    ],1))
    logit = self.logit(fuse)

    return logit, logit_pixel, logit_image
"""

In [ ]:
class Double_unit(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels, kernel_size=3, padding=1):#in_channels, middle_channels, out_channels, upscale=Upscale.upsample_bilinear):
        super().__init__()
        self.layer = nn.Sequential(
            #nn.Upsample(scale_factor=2),
            nn.Conv2d(in_channels, middle_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(middle_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        batch_size,C,H,W = x.shape
        return self.layer(x)
class Standard_unit(nn.Module):
    def __init__(self, stage, nb_filter, kernel_size=3, padding=1):#in_channels, middle_channels, out_channels, upscale=Upscale.upsample_bilinear):
        super().__init__()
        self.layer = nn.Sequential(
            #nn.Upsample(scale_factor=2),
            nn.Conv2d(nb_filter, nb_filter, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(nb_filter),
            nn.ReLU(inplace=True),
            nn.Conv2d(nb_filter, nb_filter, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(nb_filter),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        batch_size,C,H,W = x.shape
        return self.layer(x)

class Standard_unit1(nn.Module):
    def __init__(self, stage, in_c, nb_filter, kernel_size=3, padding=1):#in_channels, middle_channels, out_channels, upscale=Upscale.upsample_bilinear):
        super().__init__()
        self.layer = nn.Sequential(
            #nn.Upsample(scale_factor=2),
            nn.Conv2d(in_c, nb_filter, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(nb_filter),
            nn.ReLU(inplace=True),
            nn.Conv2d(nb_filter, nb_filter, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(nb_filter),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        batch_size,C,H,W = x.shape
        return self.layer(x)
    
class Ups(nn.Module):
    def __init__(self, in_chan, out_chan, upscale=1):#in_channels, middle_channels, out_channels, upscale=Upscale.upsample_bilinear):
        super().__init__()
        if upscale == 1:#Upscale.transposed_conv:
            self.layer = nn.Sequential(
                nn.ConvTranspose2d(in_chan, out_chan, 3, stride=2, padding=1, output_padding=1),
                nn.BatchNorm2d(out_chan),
                nn.ReLU(inplace=True)
            )
        elif upscale == 2:#Upscale.upsample_bilinear:
            self.layer = nn.Upsample(scale_factor=2,mode='bilinear')
            nn.Conv2d(in_chan, out_chan, kernel_size=1, padding=0),
            nn.BatchNorm2d(out_chan),
            nn.ReLU(inplace=True),
        else:
            self.layer = nn.PixelShuffle(upscale_factor=2)


    def forward(self, x):
        batch_size,C,H,W = x.shape
        return self.layer(x)    

"""
def standard_unit(input_tensor, stage, nb_filter, kernel_size=3):

    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, name='conv'+stage+'_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(input_tensor)
    x = Dropout(dropout_rate, name='dp'+stage+'_1')(x)
    x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, name='conv'+stage+'_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(dropout_rate, name='dp'+stage+'_2')(x)

    return x
"""
"""
Standard UNet++ [Zhou et.al, 2018]
Total params: 9,041,601
"""
class UnetPP(nn.Module):
    def __init__(self, num_class):#in_channels, middle_channels, out_channels, upscale=Upscale.upsample_bilinear):
        super().__init__()
        nb_filter = [32,64,128,256,512]
        
        self.conv1_1 = Standard_unit1(stage='11',in_c=3, nb_filter=nb_filter[0])
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2_1 = Standard_unit1(stage='21',in_c=nb_filter[0], nb_filter=nb_filter[1])
        self.pool2 = nn.MaxPool2d(2, 2)

        self.up1_2 = Ups(nb_filter[1],nb_filter[0])#Conv2DTranspose(nb_filter[0])(conv2_1)
        #self.conv1_2 = concatenate([up1_2, conv1_1], name='merge12', axis=bn_axis)
        self.conv1_2 = Standard_unit1(stage='12',in_c=nb_filter[0]+nb_filter[0], nb_filter=nb_filter[0])

        self.conv3_1 = Standard_unit1( stage='31', in_c=nb_filter[1],nb_filter=nb_filter[2])
        self.pool3 = nn.MaxPool2d(2, 2)

        self.up2_2 = Ups(nb_filter[2],nb_filter[1])#Conv2DTranspose(nb_filter[1])(conv3_1)
        #self.conv2_2 = concatenate([up2_2, conv2_1], name='merge22', axis=bn_axis)
        self.conv2_2 = Standard_unit1( stage='22', in_c=nb_filter[1]+nb_filter[1], nb_filter=nb_filter[1])

        self.up1_3 = Ups(nb_filter[1],nb_filter[0])#Conv2DTranspose(nb_filter[0])(conv2_2)
        #self.conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13', axis=bn_axis)
        self.conv1_3 = Standard_unit1( stage='13', in_c=nb_filter[0]*3, nb_filter=nb_filter[0])

        self.conv4_1 = Standard_unit1(stage='41',in_c=nb_filter[2], nb_filter=nb_filter[3])
        self.pool4 = nn.MaxPool2d(2, 2)

        self.up3_2 = Ups(nb_filter[3],nb_filter[2])#Conv2DTranspose(nb_filter[2])(conv4_1)
        #self.conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=bn_axis)
        self.conv3_2 = Standard_unit1(stage='32',in_c=nb_filter[2]+nb_filter[2], nb_filter=nb_filter[2])

        self.up2_3 = Ups(nb_filter[2],nb_filter[1])#Conv2DTranspose(nb_filter[1])(conv3_2)
        #self.conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23', axis=bn_axis)
        self.conv2_3 = Standard_unit1( stage='23', in_c=nb_filter[1]*3,nb_filter=nb_filter[1])

        self.up1_4 = Ups(nb_filter[1],nb_filter[0])#Conv2DTranspose(nb_filter[0])(conv2_3)
        #self.conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14', axis=bn_axis)
        self.conv1_4 = Standard_unit1(stage='14', in_c=nb_filter[0]*4,nb_filter=nb_filter[0])

        self.conv5_1 = Standard_unit1( stage='51',in_c=nb_filter[3], nb_filter=nb_filter[4])

        self.up4_2 = Ups(nb_filter[4],nb_filter[3])#Conv2DTranspose(nb_filter[3])(conv5_1)
        #self.conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=bn_axis)
        self.conv4_2 = Standard_unit1(stage='42', in_c=nb_filter[3]+nb_filter[3],nb_filter=nb_filter[3])

        self.up3_3 = Ups(nb_filter[3],nb_filter[2])#Conv2DTranspose(nb_filter[2])(conv4_2)
        #self.conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=bn_axis)
        self.conv3_3 = Standard_unit1( stage='33', in_c=nb_filter[2]*3,nb_filter=nb_filter[2])

        self.up2_4 = Ups(nb_filter[2],nb_filter[1])#Conv2DTranspose(nb_filter[1])(conv3_3)
        #self.conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24', axis=bn_axis)
        self.conv2_4 = Standard_unit1(stage='24',in_c=nb_filter[1]*4, nb_filter=nb_filter[1])

        self.up1_5 = Ups(nb_filter[1],nb_filter[0])#Conv2DTranspose(nb_filter[0])(conv2_4)
        #self.conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15', axis=bn_axis)
        self.conv1_5 = Standard_unit1(stage='15', in_c=nb_filter[0]*5,nb_filter=nb_filter[0])        
        
        self.nestnet_output_1 = nn.Conv2d(nb_filter[0],num_class, 1, padding=0)#(conv1_2)
        self.nestnet_output_2 = nn.Conv2d(nb_filter[0],num_class, 1, padding=0)#(conv1_3)
        self.nestnet_output_3 = nn.Conv2d(nb_filter[0],num_class, 1, padding=0)#(conv1_4)
        self.nestnet_output_4 = nn.Conv2d(nb_filter[0],num_class, 1, padding=0)#(conv1_5)        
        
        #= nn.Conv2d(num_class,num_class, 1, padding=0)#
    def forward(self, img_input):#, img_rows, img_cols, color_type=1, num_class=1, deep_supervision=False):
        img_input=img_input.float()
        #nb_filter = [32,64,128,256,512]

        # Handle Dimension Ordering for different backends
        #global bn_axis
        #if K.image_dim_ordering() == 'tf':
        #  bn_axis = 3
        #  img_input = Input(shape=(img_rows, img_cols, color_type), name='main_input')
        #else:
        #  bn_axis = 1
        #  img_input = Input(shape=(color_type, img_rows, img_cols), name='main_input')
        bn_axis = 1
        
        conv1_1 = self.conv1_1(img_input)
        pool1 = self.pool1(conv1_1)

        conv2_1 = self.conv2_1(pool1)
        pool2 = self.pool2(conv2_1)

        up1_2 = self.up1_2(conv2_1)
        conv1_2 = torch.cat([up1_2, conv1_1],  bn_axis)
        conv1_2 = self.conv1_2(conv1_2)

        conv3_1 = self.conv3_1(pool2)
        pool3 = self.pool3(conv3_1)

        up2_2 = self.up2_2(conv3_1)
        conv2_2 = torch.cat([up2_2, conv2_1],  bn_axis)
        conv2_2 = self.conv2_2(conv2_2)

        up1_3 = self.up1_3(conv2_2)
        conv1_3 = torch.cat([up1_3, conv1_1, conv1_2],  bn_axis)
        conv1_3 = self.conv1_3(conv1_3)

        conv4_1 = self.conv4_1(pool3)
        pool4 = self.pool4(conv4_1)

        up3_2 = self.up3_2(conv4_1)
        conv3_2 = torch.cat([up3_2, conv3_1],  bn_axis)
        conv3_2 = self.conv3_2(conv3_2)

        up2_3 = self.up2_3(conv3_2)
        conv2_3 = torch.cat([up2_3, conv2_1, conv2_2],  bn_axis)
        conv2_3 = self.conv2_3(conv2_3)

        up1_4 = self.up1_4(conv2_3)
        conv1_4 = torch.cat([up1_4, conv1_1, conv1_2, conv1_3], bn_axis)
        conv1_4 = self.conv1_4(conv1_4)

        conv5_1 = self.conv5_1(pool4)

        up4_2 = self.up4_2(conv5_1)
        conv4_2 = torch.cat([up4_2, conv4_1],  bn_axis)
        conv4_2 = self.conv4_2(conv4_2)

        up3_3 = self.up3_3(conv4_2)
        conv3_3 = torch.cat([up3_3, conv3_1, conv3_2],  bn_axis)
        conv3_3 = self.conv3_3(conv3_3)

        up2_4 = self.up2_4(conv3_3)
        conv2_4 = torch.cat([up2_4, conv2_1, conv2_2, conv2_3],  bn_axis)
        conv2_4 = self.conv2_4(conv2_4)

        up1_5 = self.up1_5(conv2_4)
        conv1_5 = torch.cat([up1_5, conv1_1, conv1_2, conv1_3, conv1_4],  bn_axis)
        conv1_5 = self.conv1_5(conv1_5)
    
        nestnet_output_1 = self.nestnet_output_1(conv1_2)
        nestnet_output_2 = self.nestnet_output_2(conv1_3)
        nestnet_output_3 = self.nestnet_output_3(conv1_4)
        nestnet_output_4 = self.nestnet_output_4(conv1_5)
        """
        if deep_supervision:
            model = Model(input=img_input, output=[nestnet_output_1,
                                                   nestnet_output_2,
                                                   nestnet_output_3,
                                                   nestnet_output_4])
        else:
            model = Model(input=img_input, output=[nestnet_output_4])
        """
        return nestnet_output_4, nestnet_output_3, nestnet_output_2, nestnet_output_1

    
    
    
    
class LinkNet34deeps(nn.Module):
    def __init__(self, num_classes, num_channels=3):
        super().__init__()
        assert num_channels == 3, "num channels not used now. to use changle first conv layer to support num channels other then 3"
        filters = [64, 128, 256, 512]
        resnet = models.resnet34(pretrained=True)

        self.firstconv = resnet.conv1
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3
        self.encoder4 = resnet.layer4

        # Decoder
        self.decoder4 = DecoderBlock(filters[3], filters[2])
        self.conv4=nn.Sequential(
            nn.Conv2d(512,256,1),
            nn.BatchNorm2d(256),
            nonlinearity(inplace=True),
        ) #nn.Conv2d(512,256,1) 
        self.conv44f=nn.Conv2d(64,1,1)
        self.conv44=nn.Sequential(
            nn.Conv2d(256, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nonlinearity(inplace=True),
            #nn.Conv2d(64, 1, 1),
        ) 
        self.decoder3 = DecoderBlock(filters[2], filters[1])
        self.conv3=nn.Sequential(
            nn.Conv2d(256,128,1),
            nn.BatchNorm2d(128),
            nonlinearity(inplace=True),
        ) #nn.Conv2d(256,128,1) 
        self.conv33f=nn.Conv2d(64,1,1)
        self.conv33= nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nonlinearity(inplace=True),
            #nn.Conv2d(64, 1, 1),
        ) 
        self.decoder2 = DecoderBlock(filters[1], filters[0])
        self.conv2 = nn.Sequential(
            nn.Conv2d(128,64,1),
            nn.BatchNorm2d(64),
            nonlinearity(inplace=True),
        ) #nn.Conv2d(128,64,1)  
        self.conv22f=nn.Conv2d(64,1,1)
        self.conv22=nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nonlinearity(inplace=True),
            #nn.Conv2d(64, 1, 1),
        ) 
        self.decoder1 = DecoderBlock(filters[0], filters[0])

        # Final Classifier
        self.finaldeconv1 = ConvUp(filters[0], filters[0])
        
        # Final Classifier
        self.logit = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nonlinearity(inplace=True),
            nn.Conv2d(64, 1, 1),
        )        
        self.logit_image = nn.Sequential(
            #nn.Linear(512, 128),
            #nn.ReLU(inplace=True),
            #nn.Linear(128, 1),
            nn.Linear(64, 1),
        )
        self.fuse_image = nn.Sequential(
            nn.Linear(512, 64),
            #nn.ReLU(inplace=True),
            #nn.Linear(128, 1),
        )

    # noinspection PyCallingNonCallable
    def forward(self, x):
        # Encoder
        batch_size,dim,m,n=x.shape
        x = x.float()
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        #x = self.firstmaxpool(x)
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)

        d4 = torch.cat([self.decoder4(e4) , e3], 1)#concat([self.decoder5(e5) , e4])
        d4 = self.conv4(d4)
        f4 = self.conv44(d4)
        # d4 = e3
        #d3 = self.decoder3(d4) + e2
        #print(e2.shape)
        d3 = torch.cat([self.decoder3(d4) , e2], 1)#concat([self.decoder5(e5) , e4])
        #print(d3.shape)
        d3 = self.conv3(d3)
        f3 = self.conv33(d3)
        
        #d2 = self.decoder2(d3) + e1
        d2 = torch.cat([self.decoder2(d3) , e1], 1)#concat([self.decoder5(e5) , e4])
        d2 = self.conv2(d2)
        f2 = self.conv22(d2)
        
        #d1 = self.decoder1(d2)

        # Final Classification
        f = self.finaldeconv1(d2)
        #f = self.finalrelu1(f)
        #f = self.logit(f)
          
        f21 = F.upsample(f2,scale_factor= 2, mode='bilinear',align_corners=False)
        f22 = self.conv22f(f21)  
        
        f31 = F.upsample(f3,scale_factor= 4, mode='bilinear',align_corners=False)
        f33 = self.conv33f(f31)  
        
        f41 = F.upsample(f4,scale_factor= 8, mode='bilinear',align_corners=False)
        f44 = self.conv44f(f41)  

        e = F.adaptive_avg_pool2d(e4, output_size=1).view(batch_size,-1) #image pool#-512-1-1
        e = F.dropout(e, p=0.50, training=self.training)#
        fuse_image  = self.fuse_image(e)#-64-1-1
        logit_image = self.logit_image(fuse_image).view(-1)#-1-1-1

        fuse_pixel=f+f21+f31+f41
        
        fuse = torch.cat([ #fuse
            fuse_pixel,
            F.upsample(fuse_image.view(batch_size,-1,1,1,),scale_factor=m, mode='nearest')
        ],1)     
        
               
        f = self.logit(fuse)   
        
        return f, f22, f33, f44, logit_image
    
        """
        self.fuse_pixel  = nn.Sequential(        
            nn.Conv2d(320, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.logit_pixel  = nn.Sequential(
            #nn.Conv2d(320, 64, kernel_size=3, padding=1),
            #nn.ReLU(inplace=True),
            nn.Conv2d( 64,  1, kernel_size=1, padding=0),
        )

        self.logit_image = nn.Sequential(
            #nn.Linear(512, 128),
            #nn.ReLU(inplace=True),
            #nn.Linear(128, 1),
            nn.Linear(64, 1),
        )
        self.fuse_image = nn.Sequential(
            nn.Linear(512, 64),
            #nn.ReLU(inplace=True),
            #nn.Linear(128, 1),
        )
        self.fuse = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.logit = nn.Sequential(
            #nn.Conv2d(128, 64, kernel_size=3, padding=1),
            #nn.ReLU(inplace=True),
            nn.Conv2d( 64,  1, kernel_size=1, padding=0),
        )    
        """
    
    
    
def criterion(output_4, output_3, output_2, output_1, logit_image, truth_pixel, is_average=True):
    #print(torch.sum(torch.sum(truth_pixel)))
    bat,dim,m,n=truth_pixel.shape

    truth_image = torch.Tensor(bat)
    i=0
    for x in truth_pixel:
        truth_image[i]=(x.squeeze().sum()>0).cuda().float()
        i=i+1

    truth_image=truth_image.type(torch.cuda.FloatTensor)

    loss_image = F.binary_cross_entropy_with_logits(logit_image, truth_image, reduce=is_average)#1-1-1
    
    loss_logit1 = F.binary_cross_entropy_with_logits(output_1, truth_pixel)#1-1-1
    loss_logit2 = F.binary_cross_entropy_with_logits(output_2, truth_pixel)#1-1-1
    loss_logit3 = F.binary_cross_entropy_with_logits(output_3, truth_pixel)#1-1-1
    loss_logit4 = F.binary_cross_entropy_with_logits(output_4, truth_pixel)#1-1-1

    #weight_image, weight_pixel = 0.1, 10  #focal
    weight_1, weight_2, weight_3, weight_4, weight_image = 0.2, 0.2, 0.2, 1, 0.05  #lovasz?
    #weight_image, weight_pixel = 0.1, 2 #bce



    return weight_1*loss_logit1+ weight_2*loss_logit2+weight_3*loss_logit3+weight_4*loss_logit4+weight_image*loss_image

In [ ]:
model = LinkNet34deeps(1) #

In [ ]:
#print(model)
params = list(model.parameters())
k = 0
for i in params:
    l = 1
    print("该层的结构：" + str(list(i.size())))
    for j in i.size():
        l *= j
    print("该层参数和：" + str(l))
    k = k + l
print("总参数数量和：" + str(k))

## Loss function
Due to non-differentiability of the competition evaluation metric (F2 Score at different intersection over union (IoU) thresholds) we use combination of  the BinnaryCrossEntropy loss and minus Jaccard index (https://en.wikipedia.org/wiki/Jaccard_index)


In [ ]:
   """
    def set_loss(self):
        if self.activation_func == 'softmax':
            loss_function = partial(mixed_dice_cross_entropy_loss,
                                    dice_loss=multiclass_dice_loss,
                                    cross_entropy_loss=nn.CrossEntropyLoss(),
                                    dice_activation='softmax',
                                    dice_weight=self.architecture_config['model_params']['dice_weight'],
                                    cross_entropy_weight=self.architecture_config['model_params']['bce_weight']
                                    )
        elif self.activation_func == 'sigmoid':
            loss_function = partial(mixed_dice_bce_loss,
                                    dice_loss=multiclass_dice_loss,
                                    bce_loss=nn.BCEWithLogitsLoss(),
                                    dice_activation='sigmoid',
                                    dice_weight=self.architecture_config['model_params']['dice_weight'],
                                    bce_weight=self.architecture_config['model_params']['bce_weight']
                                    )
        else:
            raise Exception('Only softmax and sigmoid activations are allowed')
        self.loss_function = [('mask', loss_function, 1.0)]
"""
class DiceLoss(nn.Module):
    def __init__(self, smooth=0, eps=1e-7):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
        self.eps = eps

    def forward(self, output, target):
        return 1 - (2 * torch.sum(output * target) + self.smooth) / (
                torch.sum(output) + torch.sum(target) + self.smooth + self.eps)


def mixed_dice_bce_loss(output, target, dice_weight=0.2, dice_loss=None,
                        bce_weight=0.9, bce_loss=None,
                        smooth=0, dice_activation='sigmoid'):
    num_classes = output.size(1)
    target = target[:, :num_classes, :, :].long()
    if bce_loss is None:
        bce_loss = nn.BCEWithLogitsLoss()
    if dice_loss is None:
        dice_loss = multiclass_dice_loss
    return dice_weight * dice_loss(output, target, smooth, dice_activation) + bce_weight * bce_loss(output, target)


def mixed_dice_cross_entropy_loss(output, target, dice_weight=0.5, dice_loss=None,
                                  cross_entropy_weight=0.5, cross_entropy_loss=None, smooth=0,
                                  dice_activation='softmax'):
    num_classes_without_background = output.size(1) - 1
    dice_output = output[:, 1:, :, :]
    dice_target = target[:, :num_classes_without_background, :, :].long()
    cross_entropy_target = torch.zeros_like(target[:, 0, :, :]).long()
    for class_nr in range(num_classes_without_background):
        cross_entropy_target = where(target[:, class_nr, :, :], class_nr + 1, cross_entropy_target)
    if cross_entropy_loss is None:
        cross_entropy_loss = nn.CrossEntropyLoss()
    if dice_loss is None:
        dice_loss = multiclass_dice_loss
    return dice_weight * dice_loss(dice_output, dice_target, smooth,
                                   dice_activation) + cross_entropy_weight * cross_entropy_loss(output,
                                                                                                cross_entropy_target)


def multiclass_dice_loss(output, target, smooth=0, activation='softmax'):
    """Calculate Dice Loss for multiple class output.
    Args:
        output (torch.Tensor): Model output of shape (N x C x H x W).
        target (torch.Tensor): Target of shape (N x H x W).
        smooth (float, optional): Smoothing factor. Defaults to 0.
        activation (string, optional): Name of the activation function, softmax or sigmoid. Defaults to 'softmax'.
    Returns:
        torch.Tensor: Loss value.
    """
    if activation == 'softmax':
        activation_nn = torch.nn.Softmax2d()
    elif activation == 'sigmoid':
        activation_nn = torch.nn.Sigmoid()
    else:
        raise NotImplementedError('only sigmoid and softmax are implemented')

    loss = 0
    dice = DiceLoss(smooth=smooth)
    output = activation_nn(output)
    num_classes = output.size(1)
    target.data = target.data.float()
    for class_nr in range(num_classes):
        loss += dice(output[:, class_nr, :, :], target[:, class_nr, :, :])
    return loss / num_classes


def where(cond, x_1, x_2):
    cond = cond.long()
    return (cond * x_1) + ((1 - cond) * x_2)

In [ ]:
from torchvision import models
import torchvision
res34=torchvision.models.resnet34(pretrained=True)
print(res34)

In [ ]:
 class LossBinary:
    """
     Implementation from  https://github.com/ternaus/robot-surgery-segmentation
    """

    def __init__(self, jaccard_weight=0):
        self.nll_loss = nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        loss = self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            eps = 1e-15
            jaccard_target = (targets == 1.0).float()
            jaccard_output = F.sigmoid(outputs)

            intersection = (jaccard_output * jaccard_target).sum()
            union = jaccard_output.sum() + jaccard_target.sum()

            loss -= self.jaccard_weight * torch.log((intersection + eps) / (union - intersection + eps))
        return loss



## Validation routine

In [ ]:
def validation(model: nn.Module, criterion, valid_loader):
    print("Validation on hold-out....")
    model.eval()
    losses = []
    jaccard = []
    ious = []
    for inputs, targets in valid_loader:
        inputs = variable(inputs, volatile=True)
        targets = variable(targets)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        losses.append(loss.data[0])
        jaccard += [get_jaccard(targets, (outputs > 0).float()).data[0]]
        ious += [get_iou_vector(targets, (outputs > 0).float())]
    
    valid_ious = np.mean(ious)
    
    valid_loss = np.mean(losses)  # type: float

    valid_jaccard = np.mean(jaccard)

    print('Valid loss: {:.5f}, jaccard: {:.5f}, ious: {:.5f}'.format(valid_loss, valid_jaccard, valid_ious))
    metrics = {'valid_loss': valid_loss, 'jaccard_loss': valid_jaccard, 'ious_loss': valid_ious}
    return metrics
def validation4(model: nn.Module, criterion, valid_loader):
    print("Validation on hold-out....")
    model.eval()
    losses = []
    jaccard = []
    ious = []
    for inputs, targets in valid_loader:
        inputs = variable(inputs, volatile=True)
        targets = variable(targets)
        outputs,o2,o3,o4= model(inputs)
        loss = criterion(outputs,o2,o3,o4, targets)
        losses.append(loss.data[0])
        jaccard += [get_jaccard(targets, (outputs > 0).float()).data[0]]
        ious += [get_iou_vector(targets, (outputs > 0).float())]
    
    valid_ious = np.mean(ious)
    
    valid_loss = np.mean(losses)  # type: float

    valid_jaccard = np.mean(jaccard)

    print('Valid loss: {:.5f}, jaccard: {:.5f}, ious: {:.5f}'.format(valid_loss, valid_jaccard, valid_ious))
    metrics = {'valid_loss': valid_loss, 'jaccard_loss': valid_jaccard, 'ious_loss': valid_ious}
    return metrics
def validation5(model: nn.Module, criterion, valid_loader):
    print("Validation on hold-out....")
    model.eval()
    losses = []
    jaccard = []
    ious = []
    for inputs, targets in valid_loader:
        inputs = variable(inputs, volatile=True)
        targets = variable(targets)
        outputs,o2,o3,o4,o5= model(inputs)
        loss = criterion(outputs,o2,o3,o4,o5, targets)
        losses.append(loss.data[0])
        jaccard += [get_jaccard(targets, (outputs > 0).float()).data[0]]
        ious += [get_iou_vector(targets, (outputs > 0).float())]
    
    valid_ious = np.mean(ious)
    
    valid_loss = np.mean(losses)  # type: float

    valid_jaccard = np.mean(jaccard)

    print('Valid loss: {:.5f}, jaccard: {:.5f}, ious: {:.5f}'.format(valid_loss, valid_jaccard, valid_ious))
    metrics = {'valid_loss': valid_loss, 'jaccard_loss': valid_jaccard, 'ious_loss': valid_ious}
    return metrics
def get_jaccard(y_true, y_pred):
    epsilon = 1e-15
    intersection = (y_pred * y_true).sum(dim=-2).sum(dim=-1).sum(dim = -1)
    union = y_true.sum(dim=-2).sum(dim=-1).sum(dim=-1) + y_pred.sum(dim=-2).sum(dim=-1).sum(dim = -1)

    return (intersection / (union - intersection + epsilon)).mean()

In [ ]:
import numpy as np # linear algebra

def get_iou_vector_single(A, B):
    #A:True,B:Pred
    #print(A.shape)
    batch_size = A.shape[0]
    print(batch_size)
    #metric = []
    #for batch in range(batch_size):
    t, p = A.squeeze(), B.squeeze()
    if np.count_nonzero(t) == 0 and np.count_nonzero(p) > 0:
        metric=0
        return np.mean(metric)
        #continue
    if np.count_nonzero(t) == 0 and np.count_nonzero(p) == 0:
        metric=1
        return np.mean(metric)
        #continue

    iou = jaccard(t, p)
    #print(iou)
    thresholds = np.arange(0.5, 1, 0.05)
    s = []
    for thresh in thresholds:
        #s.append(iou > thresh)
        s.append(iou > thresh)
    metric=np.mean(s)
    #print(metric.shape)
    return np.mean(metric)

def get_iou_vector(A, B):
    #A:True,B:Pred
    #print(A.shape)
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch].squeeze(), B[batch].squeeze()
        if np.count_nonzero(t) == 0 and np.count_nonzero(p) > 0:
            metric.append(0)
            continue
        if np.count_nonzero(t) == 0 and np.count_nonzero(p) == 0:
            metric.append(1)
            continue

        iou = jaccard(t, p)
        #print(iou)
        thresholds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in thresholds:
            #s.append(iou > thresh)
            s.append(iou > thresh)
        metric.append(np.mean(s))
    #print(metric.shape)
    return np.mean(metric)



def jaccard(y_true, y_pred):
    epsilon = 1e-15
    intersection = (y_pred * y_true).sum().sum()
    union = y_true.sum().sum() + y_pred.sum().sum()

    return ((intersection + epsilon)/ (union - intersection + epsilon)).mean()

#thresholds = np.linspace(0, 1, 50)
#ious = np.array([get_iou_vector(y_valid_ori, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

#json_1 = {'num':1112, 'date':datetime.now()}
#print(json.dumps(json_1, cls=MyEncoder))

In [ ]:
# sume helper functions
def variable(x, volatile=False):
    if isinstance(x, (list, tuple)):
        return [variable(y, volatile=volatile) for y in x]
    return cuda(Variable(x, volatile=volatile))

def cuda(x):
    return x.cuda(async=True) if torch.cuda.is_available() else x

def write_event(log, lr, step: int, **data):
    data['lr'] = lr
    data['step'] = step
    data['dt'] = datetime.now().isoformat()
    log.write(json.dumps(data, sort_keys=True, cls=MyEncoder))
    log.write('\n')
    log.flush()

# ****LOSS********

In [ ]:
from __future__ import print_function, division

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
try:
    from itertools import  ifilterfalse
except ImportError: # py3k
    from itertools import  filterfalse


def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.float().cumsum(0)
    union = gts + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


def iou_binary(preds, labels, EMPTY=1., ignore=None, per_image=True):
    """
    IoU for foreground class
    binary: 1 foreground, 0 background
    """
    if not per_image:
        preds, labels = (preds,), (labels,)
    ious = []
    for pred, label in zip(preds, labels):
        intersection = ((label == 1) & (pred == 1)).sum()
        union = ((label == 1) | ((pred == 1) & (label != ignore))).sum()
        if not union:
            iou = EMPTY
        else:
            iou = float(intersection) / union
        ious.append(iou)
    iou = mean(ious)    # mean accross images if per_image
    return 100 * iou


def iou(preds, labels, C, EMPTY=1., ignore=None, per_image=False):
    """
    Array of IoU for each (non ignored) class
    """
    if not per_image:
        preds, labels = (preds,), (labels,)
    ious = []
    for pred, label in zip(preds, labels):
        iou = []    
        for i in range(C):
            if i != ignore: # The ignored label is sometimes among predicted classes (ENet - CityScapes)
                intersection = ((label == i) & (pred == i)).sum()
                union = ((label == i) | ((pred == i) & (label != ignore))).sum()
                if not union:
                    iou.append(EMPTY)
                else:
                    iou.append(float(intersection) / union)
        ious.append(iou)
    ious = map(mean, zip(*ious)) # mean accross images if per_image
    return 100 * np.array(ious)


# --------------------------- BINARY LOSSES ---------------------------


def lovasz_hinge(logits, labels, per_image=False, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * signs)#Variable(signs))############
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    #loss = torch.dot(F.elu(errors_sorted)+1,  grad)#Variable(grad))#ELU+1
    loss = torch.dot(F.relu(errors_sorted),  grad)#Variable(grad))#RELU
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels


class StableBCELoss(torch.nn.modules.Module):
    def __init__(self):
         super(StableBCELoss, self).__init__()
    def forward(self, input, target):
         neg_abs = - input.abs()
         loss = input.clamp(min=0) - input * target + (1 + neg_abs.exp()).log()
         return loss.mean()


def binary_xloss(logits, labels, ignore=None):
    """
    Binary Cross entropy loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      ignore: void class id
    """
    logits, labels = flatten_binary_scores(logits, labels, ignore)
    loss = StableBCELoss()(logits, Variable(labels.float()))
    return loss


# --------------------------- MULTICLASS LOSSES ---------------------------


def lovasz_softmax(probas, labels, only_present=False, per_image=False, ignore=None):
    """
    Multi-class Lovasz-Softmax loss
      probas: [B, C, H, W] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [B, H, W] Tensor, ground truth labels (between 0 and C - 1)
      only_present: average only on classes present in ground truth
      per_image: compute the loss per image instead of per batch
      ignore: void class labels
    """
    if per_image:
        loss = mean(lovasz_softmax_flat(*flatten_probas(prob.unsqueeze(0), lab.unsqueeze(0), ignore), only_present=only_present)
                          for prob, lab in zip(probas, labels))
    else:
        loss = lovasz_softmax_flat(*flatten_probas(probas, labels, ignore), only_present=only_present)
    return loss


def lovasz_softmax_flat(probas, labels, only_present=False):
    """
    Multi-class Lovasz-Softmax loss
      probas: [P, C] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [P] Tensor, ground truth labels (between 0 and C - 1)
      only_present: average only on classes present in ground truth
    """
    C = probas.size(1)
    losses = []
    for c in range(C):
        fg = (labels == c).float() # foreground for class c
        if only_present and fg.sum() == 0:
            continue
        errors = (fg - probas[:, c]).abs()#(Variable(fg) - probas[:, c]).abs()#############################
        errors_sorted, perm = torch.sort(errors, 0, descending=True)
        perm = perm.data
        fg_sorted = fg[perm]
        losses.append(torch.dot(errors_sorted, lovasz_grad(fg_sorted)))#Variable(lovasz_grad(fg_sorted))))
    return mean(losses)


def flatten_probas(probas, labels, ignore=None):
    """
    Flattens predictions in the batch
    """
    B, C, H, W = probas.size()
    probas = probas.permute(0, 2, 3, 1).contiguous().view(-1, C)  # B * H * W, C = P, C
    labels = labels.view(-1)
    if ignore is None:
        return probas, labels
    valid = (labels != ignore)
    vprobas = probas[valid.nonzero().squeeze()]
    vlabels = labels[valid]
    return vprobas, vlabels

def xloss(logits, labels, ignore=None):
    """
    Cross entropy loss
    """
    return F.cross_entropy(logits, Variable(labels), ignore_index=255)


# --------------------------- HELPER FUNCTIONS ---------------------------

def mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(np.isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n

In [ ]:
def lovasz_loss(outputs,targets):
    return lovasz_hinge(outputs,targets)
#lovasz_softmax(torch.from_numpy(outputs.data.numpy()),torch.from_numpy(targets.data.numpy()))

class LossBinary1:
    """
     Implementation from  https://github.com/ternaus/robot-surgery-segmentation
    """

    def __init__(self, jaccard_weight=0):
        self.nll_loss = lovasz_loss#mixed_dice_bce_loss#lovasz_loss#lovasz_softmax#nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        #print(outputs.shape)
        #print(targets.shape)
        loss = self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            eps = 1e-15
            jaccard_target = (targets == 1.0).float()
            jaccard_output = F.sigmoid(outputs)

            intersection = (jaccard_output * jaccard_target).sum()
            union = jaccard_output.sum() + jaccard_target.sum()

            loss -= self.jaccard_weight * torch.log((intersection + eps) / (union - intersection + eps))
        return loss

In [ ]:
def lovasz_hingeElu(logits, labels, per_image=False, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = mean(lovasz_hinge_flatElu(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flatElu(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flatElu(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * signs)#Variable(signs))############
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.elu(errors_sorted)+1,  grad)#Variable(grad))#ELU+1
    #loss = torch.dot(F.relu(errors_sorted),  grad)#Variable(grad))#RELU
    return loss

class LossBinaryElu:
    """
     Implementation from  https://github.com/ternaus/robot-surgery-segmentation
    """

    def __init__(self, jaccard_weight=0):
        self.nll_loss = lovasz_hingeElu#lovasz_loss#mixed_dice_bce_loss#lovasz_loss#lovasz_softmax#nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        #print(outputs.shape)
        #print(targets.shape)
        loss = self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            eps = 1e-15
            jaccard_target = (targets == 1.0).float()
            jaccard_output = F.sigmoid(outputs)

            intersection = (jaccard_output * jaccard_target).sum()
            union = jaccard_output.sum() + jaccard_target.sum()

            loss -= self.jaccard_weight * torch.log((intersection + eps) / (union - intersection + eps))
        return loss

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        # Inspired by the implementation of binary_cross_entropy_with_logits
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

        # This formula gives us the log sigmoid of 1-p if y is 0 and of p if y is 1
        invprobs = F.logsigmoid(-input * (target * 2 - 1))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()

# ****Train****

In [ ]:

# 'Cyclical Learning Rates for Training Neural Networks'- Leslie N. Smith, arxiv 2017
#       https://arxiv.org/abs/1506.01186
#       https://github.com/bckenstler/CLR

class CyclicLR():

    def __init__(self, base_lr=0.001, max_lr=0.006, step=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step = step
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: (0.5)**(x-1)
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step != None:
            self.step = new_step
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step))
        x = np.abs(self.clr_iterations/self.step - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)

    def get_rate(self, epoch=None, num_epoches=None):

        self.trn_iterations += 1
        self.clr_iterations += 1
        lr = self.clr()

        return lr

    def __str__(self):
        string = 'Cyclical Learning Rates\n' \
                + 'base_lr=%0.3f, max_lr=%0.3f'%(self.base_lr, self.max_lr)
        return string



# net ------------------------------------
# https://github.com/pytorch/examples/blob/master/imagenet/main.py ###############
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
       lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr



In [ ]:
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
       lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

In [ ]:
def save_checkpoint(state, is_best, filename='model_1.pt'):
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("=> Saving a new best")
        torch.save(state, filename) # save checkpoint
    else:
        print ("=> Validation Accuracy did not improve")

"""# Training the Model
for epoch in range(num_epochs):
 train(...) # Train
 acc = eval(...) # Evaluate after every epoch

# Some stuff with acc(accuracy)
 ...

# Get bool not ByteTensor
 is_best = bool(acc.numpy() > best_accuracy.numpy())
 # Get greater Tensor to keep track best acc
 best_accuracy = torch.FloatTensor(max(acc.numpy(), best_accuracy.numpy()))
 # Save checkpoint if is a new best
 save_checkpoint({
 'epoch': start_epoch + epoch + 1,
 'state_dict': model.state_dict(),
 'best_accuracy': best_accuracy
 }, is_best,model_path)
"""

# main train routine
# Implementation from  https://github.com/ternaus/robot-surgery-segmentation
def train(lr, model, criterion, train_loader, valid_loader, validation, init_optimizer, n_epochs=1, fold=1):
    optimizer = init_optimizer(lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=20)################
    #model = nn.DataParallel(model, device_ids=None)
    if torch.cuda.is_available():
        model.cuda()
    isrestore = False   
    startepoch = 0
    model_path = Path('model_{fold}.pt'.format(fold=fold))
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        isrestore = True
        startepoch = epoch
        
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss
    else:
        epoch = 1
        step = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))
    

    report_each = 50
    log = open('train_{fold}.log'.format(fold=fold),'at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm(total=len(train_loader) *  BATCH_SIZE)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs, targets = variable(inputs), variable(targets)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                step += 1
                tq.update(batch_size)
                losses.append(loss.data[0])
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss='{:.5f}'.format(mean_loss))
                if i and i % report_each == 0:
                    write_event(log, lr, step, loss=mean_loss)
            write_event(log, lr, step, loss=mean_loss)########################lr
            tq.close()
            #save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader)
            write_event(log, lr, step, **valid_metrics)#############################
            valid_loss = valid_metrics['valid_loss']

            acc= -valid_metrics['ious_loss']
            scheduler.step(acc)###########################
            lr=get_learning_rate(optimizer)###############################
            
            if epoch==1:
                best_accuracy=-valid_metrics['ious_loss']#valid_loss#(valid_loss.from_numpy())
                is_best=True
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
            else:
                #if isrestore == False:
                is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
                #else:
                      
                    """if epoch == startepoch:
                        best_accuracy=valid_loss
                    else:
                        is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                        best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                        if is_best:
                            print ("=> Saving a new best")
                            save(epoch) # save checkpoint
                        else:
                            print ("=> Validation Accuracy did not improve")"""
            valid_losses.append(valid_loss)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return
        
        
def train4(lr, model,train_loader, valid_loader,criterion, validation, init_optimizer, n_epochs=1, fold=1):
    optimizer = init_optimizer(lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=20)################
    #model = nn.DataParallel(model, device_ids=None)
    if torch.cuda.is_available():
        model.cuda()
    isrestore = False   
    startepoch = 0
    model_path = Path('model_{fold}.pt'.format(fold=fold))
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        isrestore = True
        startepoch = epoch
        
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss
    else:
        epoch = 1
        step = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))
    

    report_each = 50
    log = open('train_{fold}.log'.format(fold=fold),'at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm(total=len(train_loader) *  BATCH_SIZE)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs, targets = variable(inputs), variable(targets)
                outputs,o2,o3,o4 = model(inputs)
                loss = criterion(outputs, o2, o3,o4, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                step += 1
                tq.update(batch_size)
                losses.append(loss.data[0])
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss='{:.5f}'.format(mean_loss))
                if i and i % report_each == 0:
                    write_event(log, lr, step, loss=mean_loss)
            write_event(log, lr, step, loss=mean_loss)########################lr
            tq.close()
            #save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader)
            write_event(log, lr, step, **valid_metrics)#############################
            valid_loss = valid_metrics['valid_loss']

            acc= -valid_metrics['ious_loss']
            #scheduler.step(acc)###########################
            #lr=get_learning_rate(optimizer)###############################
            
            if epoch==1:
                best_accuracy=-valid_metrics['ious_loss']#valid_loss#(valid_loss.from_numpy())
                is_best=True
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
            else:
                #if isrestore == False:
                is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
                #else:
                      
                    """if epoch == startepoch:
                        best_accuracy=valid_loss
                    else:
                        is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                        best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                        if is_best:
                            print ("=> Saving a new best")
                            save(epoch) # save checkpoint
                        else:
                            print ("=> Validation Accuracy did not improve")"""
            valid_losses.append(valid_loss)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return
        
def train5(lr, model,train_loader, valid_loader,criterion, validation, init_optimizer, n_epochs=1, fold=1):
    optimizer = init_optimizer(lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=20)################
    #model = nn.DataParallel(model, device_ids=None)
    if torch.cuda.is_available():
        model.cuda()
    isrestore = False   
    startepoch = 0
    model_path = Path('model_{fold}.pt'.format(fold=fold))
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        isrestore = True
        startepoch = epoch
        
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss
    else:
        epoch = 1
        step = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))
    

    report_each = 50
    log = open('train_{fold}.log'.format(fold=fold),'at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm(total=len(train_loader) *  BATCH_SIZE)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs, targets = variable(inputs), variable(targets)
                outputs,o2,o3,o4,o5 = model(inputs)
                loss = criterion(outputs, o2, o3, o4, o5, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                step += 1
                tq.update(batch_size)
                losses.append(loss.data[0])
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss='{:.5f}'.format(mean_loss))
                if i and i % report_each == 0:
                    write_event(log, lr, step, loss=mean_loss)
            write_event(log, lr, step, loss=mean_loss)########################lr
            tq.close()
            #save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader)
            write_event(log, lr, step, **valid_metrics)#############################
            valid_loss = valid_metrics['valid_loss']

            acc= -valid_metrics['ious_loss']
            #scheduler.step(acc)###########################
            #lr=get_learning_rate(optimizer)###############################
            
            if epoch==1:
                best_accuracy=-valid_metrics['ious_loss']#valid_loss#(valid_loss.from_numpy())
                is_best=True
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
            else:
                #if isrestore == False:
                is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
                #else:
                      
                    """if epoch == startepoch:
                        best_accuracy=valid_loss
                    else:
                        is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                        best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                        if is_best:
                            print ("=> Saving a new best")
                            save(epoch) # save checkpoint
                        else:
                            print ("=> Validation Accuracy did not improve")"""
            valid_losses.append(valid_loss)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return

In [ ]:
def retrain(lr, model, criterion, train_loader, valid_loader, validation, init_optimizer, n_epochs=1, fold=1):
    optimizer = init_optimizer(lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=20)################
    #model = nn.DataParallel(model, device_ids=None)
    if torch.cuda.is_available():
        model.cuda()
    isrestore = False   
    startepoch = 0
    model_path = Path('model_{fold}.pt'.format(fold=fold))
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        isrestore = True
        startepoch = epoch
        
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss
    else:
        epoch = 2
        step = 0
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        #print(valid_loss)
        #best_accuracy=valid_loss
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss        

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))

    print ("=> Saving a new best")###############################
    save(epoch) # save checkpoint########################

    report_each = 50
    log = open('train_{fold}.log'.format(fold=fold),'at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm(total=len(train_loader) *  BATCH_SIZE)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs, targets = variable(inputs), variable(targets)
                outputs = model(inputs)
                #outputs = model(inputs)
                loss = criterion(outputs, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                step += 1
                tq.update(batch_size)
                losses.append(loss.data[0])
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss='{:.5f}'.format(mean_loss))
                if i and i % report_each == 0:
                    write_event(log, lr, step, loss=mean_loss)######################
            write_event(log, lr, step, loss=mean_loss)########################
            tq.close()
            #save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader)
            write_event(log, lr, step, **valid_metrics)##############################
            valid_loss = valid_metrics['valid_loss']

            acc= -valid_metrics['ious_loss']
            scheduler.step(acc)###########################
            lr=get_learning_rate(optimizer)###############################
            
            if epoch==1:
                best_accuracy=-valid_metrics['ious_loss']#valid_loss#(valid_loss.from_numpy())
                is_best=True
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
            else:
                #if isrestore == False:
                is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
                #else:
                      
                    """if epoch == startepoch:
                        best_accuracy=valid_loss
                    else:
                        is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                        best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                        if is_best:
                            print ("=> Saving a new best")
                            save(epoch) # save checkpoint
                        else:
                            print ("=> Validation Accuracy did not improve")"""
            valid_losses.append(valid_loss)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return

def retrain4(lr, model, train_loader, valid_loader,criterion, validation, init_optimizer, n_epochs=1, fold=1):
    optimizer = init_optimizer(lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=20)################
    #model = nn.DataParallel(model, device_ids=None)
    if torch.cuda.is_available():
        model.cuda()
    isrestore = False   
    startepoch = 0
    model_path = Path('model_{fold}.pt'.format(fold=fold))
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        isrestore = True
        startepoch = epoch
        
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss
    else:
        epoch = 2
        step = 0
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        #print(valid_loss)
        #best_accuracy=valid_loss
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss        

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))

    print ("=> Saving a new best")###############################
    save(epoch) # save checkpoint########################

    report_each = 50
    log = open('train_{fold}.log'.format(fold=fold),'at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm(total=len(train_loader) *  BATCH_SIZE)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs, targets = variable(inputs), variable(targets)
                outputs,o2,o3 ,o4= model(inputs)
                #outputs = model(inputs)
                loss = criterion(outputs, o2,o3,o4, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                step += 1
                tq.update(batch_size)
                losses.append(loss.data[0])
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss='{:.5f}'.format(mean_loss))
                if i and i % report_each == 0:
                    write_event(log, lr, step, loss=mean_loss)######################
            write_event(log, lr, step, loss=mean_loss)########################
            tq.close()
            #save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader)
            write_event(log, lr, step, **valid_metrics)##############################
            valid_loss = valid_metrics['valid_loss']

            acc= -valid_metrics['ious_loss']
            #scheduler.step(acc)###########################
            #lr=get_learning_rate(optimizer)###############################
            
            if epoch==1:
                best_accuracy=-valid_metrics['ious_loss']#valid_loss#(valid_loss.from_numpy())
                is_best=True
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
            else:
                #if isrestore == False:
                is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
                #else:
                      
                    """if epoch == startepoch:
                        best_accuracy=valid_loss
                    else:
                        is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                        best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                        if is_best:
                            print ("=> Saving a new best")
                            save(epoch) # save checkpoint
                        else:
                            print ("=> Validation Accuracy did not improve")"""
            valid_losses.append(valid_loss)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return

def retrain5(lr, model, train_loader, valid_loader,criterion, validation, init_optimizer, n_epochs=1, fold=1):
    optimizer = init_optimizer(lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=20)################
    #model = nn.DataParallel(model, device_ids=None)
    if torch.cuda.is_available():
        model.cuda()
    isrestore = False   
    startepoch = 0
    model_path = Path('model_{fold}.pt'.format(fold=fold))
    if model_path.exists():
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        isrestore = True
        startepoch = epoch
        
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss
    else:
        epoch = 2
        step = 0
        valid_metrics = validation(model, criterion, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        #print(valid_loss)
        #best_accuracy=valid_loss
        ious_loss = valid_metrics['ious_loss']
        print(ious_loss)
        best_accuracy=-ious_loss        

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
    }, str(model_path))

    print ("=> Saving a new best")###############################
    save(epoch) # save checkpoint########################

    report_each = 50
    log = open('train_{fold}.log'.format(fold=fold),'at', encoding='utf8')
    valid_losses = []
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        random.seed()
        tq = tqdm(total=len(train_loader) *  BATCH_SIZE)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs, targets = variable(inputs), variable(targets)
                outputs,o2,o3 ,o4, o5= model(inputs)
                #outputs = model(inputs)
                loss = criterion(outputs, o2,o3,o4,o5, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                step += 1
                tq.update(batch_size)
                losses.append(loss.data[0])
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss='{:.5f}'.format(mean_loss))
                if i and i % report_each == 0:
                    write_event(log, lr, step, loss=mean_loss)######################
            write_event(log, lr, step, loss=mean_loss)########################
            tq.close()
            #save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader)
            write_event(log, lr, step, **valid_metrics)##############################
            valid_loss = valid_metrics['valid_loss']

            acc= -valid_metrics['ious_loss']
            #scheduler.step(acc)###########################
            #lr=get_learning_rate(optimizer)###############################
            
            if epoch==1:
                best_accuracy=-valid_metrics['ious_loss']#valid_loss#(valid_loss.from_numpy())
                is_best=True
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
            else:
                #if isrestore == False:
                is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                if is_best:
                    print ("=> Saving a new best")
                    save(epoch) # save checkpoint
                else:
                    print ("=> Validation Accuracy did not improve")
                #else:
                      
                    """if epoch == startepoch:
                        best_accuracy=valid_loss
                    else:
                        is_best = bool(acc < best_accuracy)#(acc.numpy() > best_accuracy.numpy())
                        best_accuracy = min(acc, best_accuracy)#torch.FloatTensor(min(acc.numpy(), best_accuracy.numpy()))
                        if is_best:
                            print ("=> Saving a new best")
                            save(epoch) # save checkpoint
                        else:
                            print ("=> Validation Accuracy did not improve")"""
            valid_losses.append(valid_loss)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return

# Training......

In [ ]:
def make_loader(in_df, batch_size, shuffle=False, transform=None):
        return DataLoader(
            dataset=SaltDataset(in_df, transform=transform),
            shuffle=shuffle,
            num_workers = 0,
            batch_size = batch_size,
            pin_memory=torch.cuda.is_available()
        )

train_loader = make_loader(train_df, batch_size =  BATCH_SIZE, shuffle=True, transform=train_transform)
valid_loader = make_loader(valid_df, batch_size = BATCH_SIZE // 2, transform=None)

In [ ]:
model00 = LinkNet34deeps(1)#
model01 = LinkNet34deeps(1)#
model02 = LinkNet34deeps(1)#

model10 = LinkNet34deeps(1)#
model11 = LinkNet34deeps(1)#
model12 = LinkNet34deeps(1)#

model20 = LinkNet34deeps(1)#
model21 = LinkNet34deeps(1)#
model22 = LinkNet34deeps(1)#

model30 = LinkNet34deeps(1)#
model31 = LinkNet34deeps(1)#
model32 = LinkNet34deeps(1)#

model40 = LinkNet34deeps(1)#
model41 = LinkNet34deeps(1)#
model42 = LinkNet34deeps(1)#
#model5 = DenseNet34(1)#

model_path00 ='../input/linknet34deepsv3fulldacv0re6scalere2/model_1.pt'
model_path01 ='../input/linknet34deepsv3cv0scaleclr/model_1.pt'
model_path02 ='../input/linknet34deepsv3cv0scaleclrre/model_1.pt'

model_path10 ='../input/linknet34deepsv3fulldacv1scalere2001/model_1.pt'
model_path11 ='../input/linknet34deepsv3cv1scalereclrre2/model_1.pt'
model_path12 ='../input/linknet34deepsv3cv1scalereclrre3/model_1.pt'

model_path20 ='../input/linknet34deepsv3fulldacv2scalere2001/model_1.pt'
model_path21 ='../input/linknet34deepsv3cv2scaleclrre/model_1.pt'
model_path22 ='../input/linknet34deepsv3cv2scaleclrre3/model_1.pt'

model_path30 ='../input/linknet34deepsv3cv3scaleclrre/model_1.pt'
model_path31 ='../input/linknet34deepsv3cv3scaleclrre2/model_1.pt'
model_path32 ='../input/linknet34deepsv3cv3scaleclrre3/model_1.pt'

model_path40 ='../input/linknet34deepsv3cv4scaleclr/model_1.pt'
model_path41 ='../input/linknet34deepsv3cv4scaleclrre/model_1.pt'
model_path42 ='../input/linknet34deepsv3cv4scaleclrre3/model_1.pt'

#model_path5 ='../input/hyper0924cv5re/model_1.pt'

state00 = torch.load(str(model_path00))
state00 = {key.replace('module.', ''): value for key, value in state00['model'].items()}
model00.load_state_dict(state00)
if torch.cuda.is_available():
    model00.cuda()
model00.eval()

state01 = torch.load(str(model_path01))
state01 = {key.replace('module.', ''): value for key, value in state01['model'].items()}
model01.load_state_dict(state01)
if torch.cuda.is_available():
    model01.cuda()
model01.eval()

state02 = torch.load(str(model_path02))
state02 = {key.replace('module.', ''): value for key, value in state02['model'].items()}
model02.load_state_dict(state02)
if torch.cuda.is_available():
    model02.cuda()
model02.eval()

state10 = torch.load(str(model_path10))
state10 = {key.replace('module.', ''): value for key, value in state10['model'].items()}
model10.load_state_dict(state10)
if torch.cuda.is_available():
    model10.cuda()
model10.eval()

state11 = torch.load(str(model_path11))
state11 = {key.replace('module.', ''): value for key, value in state11['model'].items()}
model11.load_state_dict(state11)
if torch.cuda.is_available():
    model11.cuda()
model11.eval()

state12 = torch.load(str(model_path12))
state12 = {key.replace('module.', ''): value for key, value in state12['model'].items()}
model12.load_state_dict(state12)
if torch.cuda.is_available():
    model12.cuda()
model12.eval()


state20 = torch.load(str(model_path20))
state20 = {key.replace('module.', ''): value for key, value in state20['model'].items()}
model20.load_state_dict(state20)
if torch.cuda.is_available():
    model20.cuda()
model20.eval()

state21 = torch.load(str(model_path21))
state21 = {key.replace('module.', ''): value for key, value in state21['model'].items()}
model21.load_state_dict(state21)
if torch.cuda.is_available():
    model21.cuda()
model21.eval()

state22 = torch.load(str(model_path22))
state22 = {key.replace('module.', ''): value for key, value in state22['model'].items()}
model22.load_state_dict(state22)
if torch.cuda.is_available():
    model22.cuda()
model22.eval()

state30 = torch.load(str(model_path30))
state30 = {key.replace('module.', ''): value for key, value in state30['model'].items()}
model30.load_state_dict(state30)
if torch.cuda.is_available():
    model30.cuda()
model30.eval()

state31 = torch.load(str(model_path31))
state31 = {key.replace('module.', ''): value for key, value in state31['model'].items()}
model31.load_state_dict(state31)
if torch.cuda.is_available():
    model31.cuda()
model31.eval()

state32 = torch.load(str(model_path32))
state32 = {key.replace('module.', ''): value for key, value in state32['model'].items()}
model32.load_state_dict(state32)
if torch.cuda.is_available():
    model32.cuda()
model32.eval()

state40 = torch.load(str(model_path40))
state40 = {key.replace('module.', ''): value for key, value in state40['model'].items()}
model40.load_state_dict(state40)
if torch.cuda.is_available():
    model40.cuda()
model40.eval()

state41 = torch.load(str(model_path41))
state41 = {key.replace('module.', ''): value for key, value in state41['model'].items()}
model41.load_state_dict(state41)
if torch.cuda.is_available():
    model41.cuda()
model41.eval()

state42 = torch.load(str(model_path42))
state42 = {key.replace('module.', ''): value for key, value in state42['model'].items()}
model42.load_state_dict(state42)
if torch.cuda.is_available():
    model42.cuda()
model42.eval()

#train3(init_optimizer=lambda lr: torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters())),#(model.parameters(), lr=lr),
#train4(init_optimizer=lambda lr: torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-2, momentum=0.9, weight_decay=0.0001),#(model.parameters(), lr=lr),
train4(init_optimizer=lambda lr: torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters())),#(model.parameters(), lr=lr),
        lr = 5e-4,#1e-4
        n_epochs = 30,#100,#35,#40,
        model=model,
        criterion=criterion,#LossBinaryElu(jaccard_weight=0),
        train_loader=train_loader,
        valid_loader=valid_loader,
        validation=validation4,
     )

In [ ]:
def criterionL(output_4, output_3, output_2, output_1, truth_pixel, is_average=True):
    #print(torch.sum(torch.sum(truth_pixel)))
    bat,dim,m,n=truth_pixel.shape


    loss_logit1 = lovasz_loss(output_1, truth_pixel)#1-1-1
    loss_logit2 = lovasz_loss(output_2, truth_pixel)#1-1-1
    loss_logit3 = lovasz_loss(output_3, truth_pixel)#1-1-1
    loss_logit4 = lovasz_loss(output_4, truth_pixel)#1-1-1

    #weight_image, weight_pixel = 0.1, 10  #focal
    weight_1, weight_2, weight_3, weight_4 = 0.2, 0.2, 0.2, 1  #lovasz?
    #weight_image, weight_pixel = 0.1, 2 #bce



    return weight_1*loss_logit1+ weight_2*loss_logit2+weight_3*loss_logit3+weight_4*loss_logit4

def criterionL5(output_4, output_3, output_2, output_1, logit_image, truth_pixel, is_average=True):
    #print(torch.sum(torch.sum(truth_pixel)))
    bat,dim,m,n=truth_pixel.shape

    truth_image = torch.Tensor(bat)
    i=0
    for x in truth_pixel:
        truth_image[i]=(x.squeeze().sum()>0).cuda().float()
        i=i+1

    truth_image=truth_image.type(torch.cuda.FloatTensor)

    loss_image = F.binary_cross_entropy_with_logits(logit_image, truth_image, reduce=is_average)#1-1-1
    
    loss_logit1 = lovasz_loss(output_1, truth_pixel)#1-1-1
    loss_logit2 = lovasz_loss(output_2, truth_pixel)#1-1-1
    loss_logit3 = lovasz_loss(output_3, truth_pixel)#1-1-1
    loss_logit4 = lovasz_loss(output_4, truth_pixel)#1-1-1

    #weight_image, weight_pixel = 0.1, 10  #focal
    weight_1, weight_2, weight_3, weight_4, weight_image = 0.2, 0.2, 0.2, 1, 0.05  #0.1, 0.1, 0.1, 1, 0.05  #0.2, 0.2, 0.2, 1, 0.05  #lovasz?
    #weight_image, weight_pixel = 0.1, 2 #bce



    return weight_1*loss_logit1+ weight_2*loss_logit2+weight_3*loss_logit3+weight_4*loss_logit4+weight_image*loss_image

# predict and show prediction 
img,_ = valid_ds[0]
input_img = torch.unsqueeze(variable(img, volatile=True), dim=0)
mask = (F.sigmoid(model(input_img)))
out_mask = torch.squeeze(mask.data.cpu(), dim = 0)
imshow(img,out_mask)

In [ ]:
def dummy_prediction(X_test,model,mirror=True):
    #X_test0 = variable(X_test, volatile=True)
    # ...
    y_preds = model(X_test)
    y_preds = F.sigmoid(y_preds).data.cpu().numpy()
    if mirror==True:
        #print(X_test.shape)#32-3-128-128
        m_preds = model(variable(torch.FloatTensor((X_test.cpu().data.numpy()[:,:,:,::-1]).copy()), volatile=True))
        ##m_preds = model(Variable(torch.from_numpy(X_test.cpu().data.numpy()[:,:,::-1,:])))
        m_preds = F.sigmoid(variable(torch.FloatTensor((m_preds.cpu().data.numpy()[:,:,:,::-1]).copy()), volatile=True)).data.cpu().numpy()
        y_preds = 0.5 * (y_preds + m_preds)
        #y_preds = 0.5 * (y_preds + variable(torch.FloatTensor((m_preds.cpu().data.numpy()[:,:,::-1,:]).copy()), volatile=True))
    return y_preds
def dummy_prediction4(X_test,model,mirror=True):
    #X_test0 = variable(X_test, volatile=True)
    # ...
    y_preds,o2,o3,o4 = model(X_test)
    y_preds = F.sigmoid(y_preds).data.cpu().numpy()
    if mirror==True:
        #print(X_test.shape)#32-3-128-128
        m_preds,o2,o3,o4 = model(variable(torch.FloatTensor((X_test.cpu().data.numpy()[:,:,:,::-1]).copy()), volatile=True))
        ##m_preds = model(Variable(torch.from_numpy(X_test.cpu().data.numpy()[:,:,::-1,:])))
        m_preds = F.sigmoid(variable(torch.FloatTensor((m_preds.cpu().data.numpy()[:,:,:,::-1]).copy()), volatile=True)).data.cpu().numpy()
        y_preds = 0.5 * (y_preds + m_preds)
        #y_preds = 0.5 * (y_preds + variable(torch.FloatTensor((m_preds.cpu().data.numpy()[:,:,::-1,:]).copy()), volatile=True))
    return y_preds
def dummy_prediction2(X_test,model,mirror=True):
    #X_test0 = variable(X_test, volatile=True)
    # ...
    y_preds,o2,o3,o4,o5 = model(X_test)
    y_preds = F.sigmoid(y_preds).data.cpu().numpy()
    if mirror==True:
        #print(X_test.shape)#32-3-128-128
        m_preds,o2,o3,o4,o5 = model(variable(torch.FloatTensor((X_test.cpu().data.numpy()[:,:,:,::-1]).copy()), volatile=True))
        ##m_preds = model(Variable(torch.from_numpy(X_test.cpu().data.numpy()[:,:,::-1,:])))
        m_preds = F.sigmoid(variable(torch.FloatTensor((m_preds.cpu().data.numpy()[:,:,:,::-1]).copy()), volatile=True)).data.cpu().numpy()
        y_preds = 0.5 * (y_preds + m_preds)
        #y_preds = 0.5 * (y_preds + variable(torch.FloatTensor((m_preds.cpu().data.numpy()[:,:,::-1,:]).copy()), volatile=True))
    return y_preds

## Prediction for Test images

For test purposes predict only for 5000 images. Prediction of the full set of images (88500 images) takes more than 6hr !!!

test_paths = os.listdir(test_image_dir+'/images/')
print(len(test_paths), 'test images found')

train_df1=train_df1.set_index('id')

In [ ]:
#train_df1=train_df1.set_index('id')


from skimage.morphology import binary_opening, disk

loader = DataLoader(
        dataset=SaltDataset(valid_df, transform=None, mode='valid'),###########################################################
        shuffle=False,
        batch_size=BATCH_SIZE,
        num_workers=0,
        pin_memory=torch.cuda.is_available()
    ) 

y_valid_ori = []
#preds_valid = [] #
out_pred_rows = []#pd.DataFrame(columns=('id', 'mask'))#[]
for batch_num, (inputs, paths) in enumerate(tqdm(loader, desc='valid')):
    inputs = variable(inputs, volatile=True)
    #outputs = dummy_prediction2(inputs,model)
    
    outputs0 = dummy_prediction2(inputs,model00)
    outputs0 = outputs0+dummy_prediction2(inputs,model01)
    outputs0 = outputs0+dummy_prediction2(inputs,model02)
    
    outputs1 = dummy_prediction2(inputs,model10)
    outputs1 = outputs1+dummy_prediction2(inputs,model11)
    outputs1 = outputs1+dummy_prediction2(inputs,model12)
    
    outputs2 = dummy_prediction2(inputs,model20)
    outputs2 = outputs2+dummy_prediction2(inputs,model21)
    outputs2 = outputs2+dummy_prediction2(inputs,model22)
    
    outputs3 = dummy_prediction2(inputs,model30)
    outputs3 = outputs3+dummy_prediction2(inputs,model31)
    outputs3 = outputs3+dummy_prediction2(inputs,model32)
    
    outputs4 = dummy_prediction2(inputs,model40)
    outputs4 = outputs4+dummy_prediction2(inputs,model41)
    outputs4 = outputs4+dummy_prediction2(inputs,model42)
    #outputs = model(inputs)
    #y_valid_ori=[downsample(crop(F.sigmoid(outputs[i,0]).data.cpu().numpy())) for for i, image_name in enumerate(paths)]
    for i, image_name in enumerate(paths):
        #mask = outputs[i,0]#F.sigmoid(outputs[i,0]).data.cpu().numpy()
        
        mask0 = outputs0[i,0]#F.sigmoid(outputs0[i,0]).data.cpu().numpy()
        mask1 = outputs1[i,0]#F.sigmoid(outputs1[i,0]).data.cpu().numpy()
        mask2 = outputs2[i,0]#F.sigmoid(outputs2[i,0]).data.cpu().numpy()
        mask3 = outputs3[i,0]#F.sigmoid(outputs3[i,0]).data.cpu().numpy()
        mask4 = outputs4[i,0]#F.sigmoid(outputs4[i,0]).data.cpu().numpy()
        #mask5 = outputs5[i,0]#F.sigmoid(outputs5[i,0]).data.cpu().numpy()
        mask = (mask0+mask1+mask2+mask3+mask4)/15
        
        out_pred_rows.extend([restore(mask).tolist()])# += [{'id': image_name, 'mask': mask}]
        y_valid_ori.extend([train_df1.loc[image_name].masks])
        #mask = F.sigmoid(torch.FloatTensor(restore(outputs[i,0].data.cpu().numpy())))
        #out_pred_rows.extend([mask.tolist()])# += [{'id': image_name, 'mask': mask}]
        #y_valid_ori.extend([train_df1.loc[image_name].masks])

"""  if i==1:
            plt.imshow(mask)
            print(np.max(np.max(outputs[i,0].data.cpu().numpy())))
            print(mask.shape)
        cur_seg = binary_opening(mask>0.5, disk(2))
        cur_rles = multi_rle_encode(cur_seg)
        if len(cur_rles)>0:
            for c_rle in cur_rles:
                #out_pred_rows += [{'ImageId': image_name, 'EncodedPixels': c_rle}]
                out_pred_rows += [{'id': image_name, 'rle_mask': c_rle}]
        else:
            #out_pred_rows += [{'ImageId': image_name, 'EncodedPixels': None}]
            out_pred_rows += [{'id': image_name, 'rle_mask': None}]"""


In [ ]:
y_valid_ori = np.array(y_valid_ori)
preds_valid = np.array(out_pred_rows)

#out_pred_rows=pd.DataFrame(out_pred_rows)[['id', 'mask']]#=pd.DataFrame(out_pred_rows)
out_pred_rows=out_pred_rows.set_index('id')
preds_valid=np.array([out_pred_rows.loc[idx].mask for idx in ids_valid.id])
#train_df0=train_df0.set_index('id')
#y_valid_ori = np.array([train_df0.loc[idx].masks for idx in ids_valid.id])

In [ ]:
preds_valid.shape

In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)

In [ ]:
#thresholds = np.linspace(0, 1, 50)
#ious = np.array([iou_metric_batch(y_valid_ori, np.int32(preds_valid > threshold)) for threshold in tqdm(thresholds)])
thresholds = np.linspace(0, 1, 50)
ious = np.array([get_iou_vector(y_valid_ori, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

In [ ]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [ ]:
#x_test = np.array([imgexpand(np.array(load_img("../input/test/images/{}.png".format(idx), grayscale=True))) / 255 for idx in tqdm_notebook(test_df.index)]).reshape(-1, img_size_target, img_size_target, 1)

In [ ]:
test_paths = os.listdir(test_image_dir+'/images/')
print(len(test_paths), 'test images found')

In [ ]:
#test_df1 = pd.DataFrame({'id': test_paths, 'rle_mask':None})

In [ ]:
test_df0['rle_mask'] = None

In [ ]:
train_df=0
train_df0=0
train_df1=0
grp=0


In [ ]:
if not('id' in test_df0.columns.values.tolist()):
    test_df0.reset_index(inplace=True)

In [ ]:
#preds_test = model.predict(x_test)#####################################
#test_df1 = pd.DataFrame({'id': test_paths, 'rle_mask':None})
from skimage.morphology import binary_opening, disk
#test_df=test_df[:5000]
loader = DataLoader(
        dataset=SaltDataset(test_df0, transform=None, mode='predict'),
        shuffle=False,
        batch_size=BATCH_SIZE,
        num_workers=0,
        pin_memory=torch.cuda.is_available()
    ) 
preds_test = {}#[] 
#ind = []
out_pred_rows = []
for batch_num, (inputs, paths) in enumerate(tqdm(loader, desc='Predict')):
    inputs = variable(inputs, volatile=True)
    #outputs = dummy_prediction2(inputs,model)
    
    outputs0 = dummy_prediction2(inputs,model00)
    outputs0 = outputs0+dummy_prediction2(inputs,model01)
    outputs0 = outputs0+dummy_prediction2(inputs,model02)
    
    outputs1 = dummy_prediction2(inputs,model10)
    outputs1 = outputs1+dummy_prediction2(inputs,model11)
    outputs1 = outputs1+dummy_prediction2(inputs,model12)
    
    outputs2 = dummy_prediction2(inputs,model20)
    outputs2 = outputs2+dummy_prediction2(inputs,model21)
    outputs2 = outputs2+dummy_prediction2(inputs,model22)
    
    outputs3 = dummy_prediction2(inputs,model30)
    outputs3 = outputs3+dummy_prediction2(inputs,model31)
    outputs3 = outputs3+dummy_prediction2(inputs,model32)
    
    outputs4 = dummy_prediction2(inputs,model40)
    outputs4 = outputs4+dummy_prediction2(inputs,model41)
    outputs4 = outputs4+dummy_prediction2(inputs,model42)
    
    #outputs = model(inputs)
    for i, image_name in enumerate(paths):
        #mask = F.sigmoid(torch.FloatTensor(restore(outputs[i,0].data.cpu().numpy())))
        #preds_test[image_name[:-4]]=(mask)
        #mask = outputs[i,0]#F.sigmoid(outputs[i,0]).data.cpu().numpy()
        
        mask0 = outputs0[i,0]#F.sigmoid(outputs0[i,0]).data.cpu().numpy()
        mask1 = outputs1[i,0]#F.sigmoid(outputs1[i,0]).data.cpu().numpy()
        mask2 = outputs2[i,0]#F.sigmoid(outputs2[i,0]).data.cpu().numpy()
        mask3 = outputs3[i,0]#F.sigmoid(outputs3[i,0]).data.cpu().numpy()
        mask4 = outputs4[i,0]#F.sigmoid(outputs4[i,0]).data.cpu().numpy()
        #mask5 = outputs5[i,0]#F.sigmoid(outputs5[i,0]).data.cpu().numpy()
        mask = (mask0+mask1+mask2+mask3+mask4)/15
                
        if image_name[-4:]=='.png':
            preds_test[image_name[:-4]]=(restore(mask))
        else:
            preds_test[image_name]=(restore(mask))
""" cur_seg = binary_opening(mask>threshold_best, disk(2))
        cur_rles = multi_rle_encode(cur_seg)
        if len(cur_rles)>0:
            for c_rle in cur_rles:
                #out_pred_rows += [{'ImageId': image_name, 'EncodedPixels': c_rle}]
                out_pred_rows += [{'id': image_name, 'rle_mask': c_rle}]
        else:
            #out_pred_rows += [{'ImageId': image_name, 'EncodedPixels': None}]
            out_pred_rows += [{'id': image_name, 'rle_mask': None}]"""

In [ ]:
co=0
for i,idx in enumerate(test_df11.index.values):
    num=np.sum(np.sum(np.round(preds_test[idx] > threshold_best)))
    #print(num)
    if num<20 and num>0:
        co=co+1
        preds_test[idx] =np.zeros((101,101))
print(co)

In [ ]:
pred_dict = {idx: RLenc(np.round(preds_test[idx] > threshold_best)) for i, idx in enumerate(tqdm(test_df11.index.values))}#.id))}

for idx in test_df11.index.values:
    if np.sum(np.sum(np.array(pred_dict[idx],dtype = 'float_')))<20:
        pred_dict[idx]=0

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submissionpth.csv')

submission_df = pd.DataFrame(out_pred_rows)[['id', 'rle_mask']]##[['ImageId', 'EncodedPixels']]#
submission_df.to_csv('submission.csv', index=False)
submission_df#.sample(10)